<a href="https://colab.research.google.com/github/atlonxp/udemy-dl-colab/blob/master/udemy_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Mount your drive with Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
#@title Mount your drive with Google SDK
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p mydrive
!google-drive-ocamlfuse drive mydrive

from IPython.display import HTML, clear_output

clear_output()
print("Installed Successfully")

In [1]:
#@title Initial Udemy Downloader

#@markdown **Username/Password**
username = "" #@param {type:"string"}
password = "" #@param {type:"string", text:"sss"}

#@markdown **Cookies** will be used in the authentication if provided
cookies = "access_token=" #@param {type:"string"}

#@markdown **Downloader Configurations**
quality = "720"  #@param ['2160', '1440', '1080', '720', '480', '360', '240']
output = "/content/drive/My Drive/" #@param {type:"string"}

!pip install requests[security] six colorama requests unidecode pyOpenSSL

#@markdown \ 
#@markdown **Remember**: Udemy courses you would like to download must be enrolled beforehand. Otherwise, you will not be able to download those courses. 

from __future__ import unicode_literals

import codecs
import json
import itertools
import os
import re
import requests
import six
import ssl
import sys
import time
import termios
import tty
import unicodedata

from colorama import init, Fore, Back, Style
from unidecode import unidecode
from IPython.display import clear_output

# _compat.py
# -------------------------------------------------------------------------------------
import urllib.request as compat_urllib

from urllib.error import HTTPError as compat_httperr
from urllib.error import URLError as compat_urlerr
from urllib.parse import urlparse as compat_urlparse
from urllib.request import Request as compat_request
from urllib.request import urlopen as compat_urlopen
from urllib.request import build_opener as compat_opener
from html.parser import HTMLParser as compat_HTMLParser
from http.cookies import SimpleCookie as ParseCookie
from requests.exceptions import ConnectionError as conn_error

encoding, pyver = str, 3
ssl._create_default_https_context = ssl._create_unverified_context

NO_DEFAULT = object()
LOGIN_URL = 'https://www.udemy.com/join/login-popup/?displayType=ajax&display_type=popup&showSkipButton=1&returnUrlAfterLogin=https'
            #'https://www.udemy.com/api-2.0/auth/udemy-auth/login/?fields[user]=access_token'
LOGOUT_URL = 'https://www.udemy.com/user/logout'

WISHLIST_URL = "https://{portal_name}.udemy.com/api-2.0/users/me/wishlisted-courses?fields[course]=id,url,published_title&ordering=-access_time&page=1&page_size=1000"
COLLECTION_URL = "https://{portal_name}.udemy.com/api-2.0/users/me/subscribed-courses-collections/?collection_has_courses=True&course_limit=20&fields[course]=last_accessed_time,published_title&fields[user_has_subscribed_courses_collection]=@all&page=1&page_size=1000"
MY_COURSES_URL = "https://{portal_name}.udemy.com/api-2.0/users/me/subscribed-courses?fields[course]=id,url,published_title&ordering=-last_accessed,-access_time&page=1&page_size=10000"
COURSE_SEARCH = "https://{portal_name}.udemy.com/api-2.0/users/me/subscribed-courses?fields[course]=id,url,published_title&page=1&page_size=1000&ordering=-last_accessed,-access_time&search={course_name}"
COURSE_URL = 'https://{portal_name}.udemy.com/api-2.0/courses/{course_id}/cached-subscriber-curriculum-items?fields[asset]=results,external_url,time_estimation,download_urls,slide_urls,filename,asset_type,captions,stream_urls,body&fields[chapter]=object_index,title,sort_order&fields[lecture]=id,title,object_index,asset,supplementary_assets,view_html&page_size=10000'
HEADERS = {
    'Origin': 'www.udemy.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
    'Referer': 'https://www.udemy.com/join/login-popup/',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}


# _progress.py
# -------------------------------------------------------------------------------------
_spin = itertools.cycle(['-', '|', '/', '\\'])

class ProgressBar(object):

    def _spinner(self, text):
        spin = _spin.next() if pyver == 2 else _spin.__next__()
        sys.stdout.write(text + spin)
        sys.stdout.flush()
        time.sleep(0.02)

    # thanks to https://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console
    def _progress(self, iteration, total, prefix = '' , file_size='' , downloaded = '' , rate = '' ,suffix = '', bar_length = 30):
        filledLength = int(round(bar_length * iteration / float(total)))
        percents = format(100.00 * (iteration / float(total)), '.2f')
        bar = fc + sd + '#' * filledLength + fw + sd +'-' * (bar_length - filledLength)
        if '0.00' not in rate:
            sys.stdout.write('\033[2K\033[1G\r\r{}{}[{}{}*{}{}] : {}{}{}/{} {}% |{}{}{}| {} {}'.format(fc,sd,fm,sb,fc,sd,fg,sb,file_size,downloaded,percents,bar,fg,sb,rate,suffix))
            sys.stdout.flush()

    def show_progress(self, total, recvd, ratio, rate, eta):
        if total <= 1048576:
            _total_size = round(float(total) / 1024.00, 2)
            _receiving = round(float(recvd) / 1024.00, 2)
            _size = format(_total_size if _total_size < 1024.00 else _total_size/1024.00, '.2f')
            _received = format(_receiving if _receiving < 1024.00 else _receiving/1024.00,'.2f')
            suffix_size = 'KB' if _total_size < 1024.00 else 'MB'
            suffix_recvd = 'KB' if _receiving < 1024.00 else 'MB'
        else:
            _total_size = round(float(total) / 1048576, 2)
            _receiving = round(float(recvd) / 1048576, 2)
            _size = format(_total_size if _total_size < 1024.00 else _total_size/1024.00, '.2f')
            _received = format(_receiving if _receiving < 1024.00 else _receiving/1024.00,'.2f')
            suffix_size = 'MB' if _total_size < 1024.00 else 'GB'
            suffix_recvd = 'MB' if _receiving < 1024.00 else 'GB'

        _rate = round(float(rate) , 2)
        rate = format(_rate if _rate < 1024.00 else _rate/1024.00, '.2f')
        suffix_rate = 'kB/s' if _rate < 1024.00 else 'MB/s'
        (mins, secs) = divmod(eta, 60)
        (hours, mins) = divmod(mins, 60)
        if hours > 99:
            eta = "--:--:--"
        if hours == 0:
            eta = "eta %02d:%02ds" % (mins, secs)
        else:
            eta = "eta %02d:%02d:%02ds" % (hours, mins, secs)
        if secs == 0:
            eta = "\n"

        self._progress(_receiving, _total_size, file_size = str(_size) + str(suffix_size) ,\
                downloaded = str(_received) + str(suffix_recvd),\
                rate = str(rate) + str(suffix_rate),\
                suffix = str(eta),\
                bar_length = 30)


# _sanitize.py
# -------------------------------------------------------------------------------------
SLUG_OK = '-_~'

def smart_text(s, encoding='utf-8', errors='strict'):
    if isinstance(s, six.text_type):
        return s

    if not isinstance(s, six.string_types):
        if six.PY3:
            if isinstance(s, bytes):
                s = six.text_type(s, encoding, errors)
            else:
                s = six.text_type(s)
        elif hasattr(s, '__unicode__'):
            s = six.text_type(s)
        else:
            s = six.text_type(bytes(s), encoding, errors)
    else:
        s = six.text_type(s)
    return s

def slugify(s, ok=SLUG_OK, lower=True, spaces=False, only_ascii=False, space_replacement='-'):
    """
    Creates a unicode slug for given string with several options.

    L and N signify letter/number.
    http://www.unicode.org/reports/tr44/tr44-4.html#GC_Values_Table

    :param s: Your unicode string.
    :param ok: Extra characters outside of alphanumerics to be allowed.
               Default is '-_~'
    :param lower: Lower the output string. 
                  Default is True
    :param spaces: True allows spaces, False replaces a space with the "space_replacement" param
    :param only_ascii: True to replace non-ASCII unicode characters with 
                       their ASCII representations.
    :param space_replacement: Char used to replace spaces if "spaces" is False. 
                              Default is dash ("-") or first char in ok if dash not allowed
    :type s: String
    :type ok: String
    :type lower: Bool
    :type spaces: Bool
    :type only_ascii: Bool
    :type space_replacement: String
    :return: Slugified unicode string

    """

    if only_ascii and ok != SLUG_OK and hasattr(ok, 'decode'):
        try:
            ok.decode('ascii')
        except UnicodeEncodeError:
            raise ValueError(('You can not use "only_ascii=True" with '
                              'a non ascii available chars in "ok" ("%s" given)') % ok)

    rv = []
    for c in unicodedata.normalize('NFKC', smart_text(s)):
        cat = unicodedata.category(c)[0]
        if cat in 'LN' or c in ok:
            rv.append(c)
        elif cat == 'Z':  # space
            rv.append(' ')
    new = ''.join(rv).strip()

    if only_ascii:
        new = unidecode(new)
    if not spaces:
        if space_replacement and space_replacement not in ok:
            space_replacement = ok[0] if ok else ''
        new = re.sub('[%s\s]+' % space_replacement, space_replacement, new)
    if lower:
        new = new.lower()

    return new

def sanitize(title):
    _locale = {
        '194': 'A',
        '199': 'C',
        '286': 'G',
        '304': 'I',
        '206': 'I',
        '214': 'O',
        '350': 'S',
        '219': 'U',
        '226': 'a',
        '231': 'c',
        '287': 'g',
        '305': 'i',
        '238': 'i',
        '246': 'o',
        '351': 's',
        '251': 'u',
        '191': '',
        '225': 'a',
        '233': 'e',
        '237': 'i',
        '243': 'o',
        '250': 'u',
        '252': 'u',
        '168u': 'u',
        '241': 'n',
        '193': 'A',
        '201': 'E',
        '205': 'I',
        '211': 'O',
        '218': 'U',
        '220': 'U',
        '168U': 'U',
        '209': 'N',
        '223': 'ss',
    }
    _temp = ''.join([str(ord(i)) if ord(i) > 128 else i for i in title])
    for _ascii, _char in _locale.items():
        if _ascii in _temp:
            _temp = _temp.replace(_ascii, _char)

    ok = re.compile(r'[^\\/:*?"<>]')
    _title = ''.join(x if ok.match(x) else "_" for x in _temp)
    return _title


# colors.py
# -------------------------------------------------------------------------------------
init(autoreset=True)
# colors foreground text:
fc = Fore.CYAN
fg = Fore.GREEN
fw = Fore.WHITE
fr = Fore.RED
fb = Fore.BLUE
fy = Fore.YELLOW
fm = Fore.MAGENTA

# colors background text:
bc = Back.CYAN
bg = Back.GREEN
bw = Back.WHITE
br = Back.RED
bb = Back.BLUE
by = Fore.YELLOW
bm = Fore.MAGENTA

# colors style text:
sd = Style.DIM
sn = Style.NORMAL
sb = Style.BRIGHT


# banner.py
# -------------------------------------------------------------------------------------
def banner():
    banner = '''
            %s%s              __                               ____  
            %s%s   __  ______/ /__  ____ ___  __  __      ____/ / /  
            %s%s  / / / / __  / _ \/ __ `__ \/ / / /_____/ __  / /   
            %s%s / /_/ / /_/ /  __/ / / / / / /_/ /_____/ /_/ / /    
            %s%s \__,_/\__,_/\___/_/ /_/ /_/\__, /      \__,_/_/     
            %s%s                           /____/
                                            %s%sVersion : %s%s0.5
                                            %s%sAuthor  : %s%sNasir Khan (r0ot h3x49)
                                            %s%sGithub  : %s%shttps://github.com/r0oth3x49

            ''' % (fc, sb,fc, sb,fc, sb, fm, sb, fy, sb, fy, sb, fy,sd, fg, sd, fy, sd, fg, sd, fy, sd, fg, sd)
    return banner


# _getpass.py
# -------------------------------------------------------------------------------------
class GetPass(object):

    def _unix_getch(self):
        fd = sys.stdin.fileno()
        old_settings = termios.tcgetattr(fd)
        try:
            tty.setraw(sys.stdin.fileno())
            ch = sys.stdin.read(1)
        finally:
            termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
        return ch
    
    def getuser(self, prompt='Username : '):
        """Prompt for Username """
        if sys.version_info[:2] >= (3, 0):
            sys.stdout.write('{}'.format(prompt))
            sys.stdout.flush()
            username = input()
        else:
            sys.stdout.write('{}'.format(prompt))
            sys.stdout.flush()
            username = raw_input()
        return username

    def getpass(self, prompt='Password : '):
        """Prompt for password and replace each character by asterik (*)"""
        sys.stdout.write('{}'.format(prompt))
        sys.stdout.flush()
        pw = ""
        while True:
            c = _win_getch() if os.name == 'nt' else self._unix_getch()
            if os.name == 'nt':
                if ord(c) == 13:
                    break
                if ord(c) == 3:
                    raise KeyboardInterrupt
                if ord(c) == 8:
                    if len(pw) > 0:
                        pw  = pw[:-1]
                        s   = "*" * len(pw)
                        sys.stdout.write('\033[2K\033[1G')
                        sys.stdout.flush()
                        sys.stdout.write('\r\r\r{}{}'.format(prompt, s))
                        sys.stdout.flush()
                    else:
                        pass
                else:
                    if ord(c) == 27:
                        pass
                    elif ord(c) == 224:
                        c = _win_getch()
                        pass
                    else:
                        if sys.version_info[:2] >= (3, 0):
                            pw = pw + c.decode('utf-8')
                        else:
                            pw = pw + c
                        sys.stdout.write('*')
                        sys.stdout.flush()
            else:
                if ord(c) == 13:
                    break
                if ord(c) == 3:
                    raise KeyboardInterrupt
                if ord(c) == 127:
                    if len(pw) > 0:
                        pw  = pw[:-1]
                        s   = "*" * len(pw)
                        sys.stdout.write('\033[2K\033[1G')
                        sys.stdout.flush()
                        sys.stdout.write('\r\r\r{}{}'.format(prompt, s))
                        sys.stdout.flush()
                    else:
                        pass
                else:
                    if ord(c) == 27:
                        pass
                    elif ord(c) == 91 or ord(c) == 27:
                        c = self._unix_getch()
                        pass
                    else:
                        pw = pw + c
                        sys.stdout.write("*")
                        sys.stdout.flush()

        return pw


# _session.py
# -------------------------------------------------------------------------------------
class Session(object):

    def __init__(self):
        self._headers = HEADERS
        self._session = requests.sessions.Session()

    def _set_auth_headers(self, access_token='', client_id=''):
        self._headers['Authorization'] = "Bearer {}".format(access_token)
        self._headers['X-Udemy-Authorization'] = "Bearer {}".format(access_token)

    def _get(self, url):
        session = self._session.get(url, headers=self._headers)
        if session.ok or session.status_code in [502, 503]:
            return session
        if not session.ok:
            if session.status_code == 403:
                msg = {'detail': 'You should use cookie base method to authenticate or try again in few minutes'}
            else:
                msg = {'detail': ''}
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Udemy Says : %s %s %s ...\n" % (session.status_code, session.reason, msg.get('detail', '')))
            time.sleep(0.8)
            sys.exit(0)

    def _post(self, url, data, redirect=True):
        session = self._session.post(url, data, headers=self._headers, allow_redirects=redirect)
        if session.ok:
            return session
        if not session.ok:
            if session.status_code == 403:
                msg = {'detail': 'You should use cookie base method to authenticate or try again in few minutes'}
            else:
                msg = {'detail': ''}
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Udemy Says : %s %s %s ...\n" % (session.status_code, session.reason, msg.get('detail', '')))
            sys.stdout.flush()
            time.sleep(0.8)
            sys.exit(0)

    def terminate(self):
        self._set_auth_headers()
        return


# _auth.py
# -------------------------------------------------------------------------------------
class UdemyAuth(object):

    def __init__(self, username='', password=''):
        self.username = username
        self.password = password
        self._session = Session()

    def _form_hidden_input(self, form_id):
        try:
            webpage = self._session._get(LOGIN_URL).text
        except conn_error as e:
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
            time.sleep(0.8)
            sys.exit(0)
        else:
            login_form = hidden_inputs(
                            search_regex(
                                r'(?is)<form[^>]+?id=(["\'])%s\1[^>]*>(?P<form>.+?)</form>' % form_id,
                                webpage,
                                '%s form' % form_id,
                                group='form'
                                )
                            )
            login_form.update({
                'email'     : self.username,
                'password'  : self.password,
                })
            return login_form

    def authenticate(self, access_token='', client_id=''):
        if not access_token and not client_id:
            data = self._form_hidden_input(form_id='login-form')
            auth_response = self._session._post(LOGIN_URL, data=data, redirect=False)
            auth_cookies = auth_response.cookies
            
            access_token = auth_cookies.get('access_token', '')
            client_id = auth_cookies.get('client_id', '')
        
        if access_token:
            self._session._set_auth_headers(access_token=access_token, client_id=client_id)
            return self._session
        else:
            self._session._set_auth_headers()
            return None


# _utils.py
# -------------------------------------------------------------------------------------
def cache_credentials(username, password, quality="", output="", language=""):
    fname = "configuration"
    fmode = "w"
    creds = {
                "username"          : username,
                "password"          : password,
                "quality"           : quality,
                "output"            : output,
                "language"          : language
            }
    fout = open(fname, fmode)
    json.dump(creds, fout, indent=4)
    fout.close()
    return "cached"

def use_cached_credentials():
    fname = "configuration"
    try:
        fout = open(fname)
    except IOError as e:
        creds = ''
        return creds
    except Exception as e:
        creds = ''
        return creds
    else:
        creds = json.load(fout)
        fout.close()
        return creds

class HTMLAttributeParser(compat_HTMLParser):
    """Trivial HTML parser to gather the attributes for a single element"""
    def __init__(self):
        self.attrs = {}
        compat_HTMLParser.__init__(self)

    def handle_starttag(self, tag, attrs):
        self.attrs = dict(attrs)

def unescapeHTML(s):
    clean   = compat_HTMLParser()
    data    = clean.unescape(s)
    return data

def extract_attributes(html_element):
    """Given a string for an HTML element such as
    <el
         a="foo" B="bar" c="&98;az" d=boz
         empty= noval entity="&amp;"
         sq='"' dq="'"
    >
    Decode and return a dictionary of attributes.
    {
        'a': 'foo', 'b': 'bar', c: 'baz', d: 'boz',
        'empty': '', 'noval': None, 'entity': '&',
        'sq': '"', 'dq': '\''
    }.
    NB HTMLParser is stricter in Python 2.6 & 3.2 than in later versions,
    but the cases in the unit test will work for all of 2.6, 2.7, 3.2-3.5.
    """
    parser = HTMLAttributeParser()
    try:
        parser.feed(html_element)
        parser.close()
    except compat_HTMLParseError:
        pass
    return parser.attrs

def hidden_inputs(html):
    html = re.sub(r'<!--(?:(?!<!--).)*-->', '', html)
    hidden_inputs = {}
    for input in re.findall(r'(?i)(<input[^>]+>)', html):
        attrs = extract_attributes(input)
        if not input:
            continue
        if attrs.get('type') not in ('hidden', 'submit'):
            continue
        name = attrs.get('name') or attrs.get('id')
        value = attrs.get('value')
        if name and value is not None:
            hidden_inputs[name] = value
    return hidden_inputs

def search_regex(pattern, string, name, default=NO_DEFAULT, fatal=True, flags=0, group=None):
    """
    Perform a regex search on the given string, using a single or a list of
    patterns returning the first matching group.
    In case of failure return a default value or raise a WARNING or a
    RegexNotFoundError, depending on fatal, specifying the field name.
    """
    if isinstance(pattern, str):
        mobj = re.search(pattern, string, flags)
    else:
        for p in pattern:
            mobj = re.search(p, string, flags)
            if mobj:
                break

    _name = name

    if mobj:
        if group is None:
            # return the first matching group
            return next(g for g in mobj.groups() if g is not None)
        else:
            return mobj.group(group)
    elif default is not NO_DEFAULT:
        return default
    elif fatal:
        print('[-] Unable to extract %s' % _name)
        exit(0)
    else:
        print('[-] unable to extract %s' % _name)
        exit(0)

def parse_json(json_string, video_id, transform_source=None, fatal=True):
    if transform_source:
        json_string = transform_source(json_string)
    try:
        return json.loads(json_string)
    except ValueError as ve:
        errmsg = '[-] %s: Failed to parse JSON ' % video_id
        if fatal:
            print(errmsg, ve)
        else:
            print(errmsg + str(ve))

def js_to_json(code):
    COMMENT_RE = r'/\*(?:(?!\*/).)*?\*/|//[^\n]*'
    SKIP_RE = r'\s*(?:{comment})?\s*'.format(comment=COMMENT_RE)
    INTEGER_TABLE = (
        (r'(?s)^(0[xX][0-9a-fA-F]+){skip}:?$'.format(skip=SKIP_RE), 16),
        (r'(?s)^(0+[0-7]+){skip}:?$'.format(skip=SKIP_RE), 8),
    )

    def fix_kv(m):
        v = m.group(0)
        if v in ('true', 'false', 'null'):
            return v
        elif v.startswith('/*') or v.startswith('//') or v == ',':
            return ""

        if v[0] in ("'", '"'):
            v = re.sub(r'(?s)\\.|"', lambda m: {
                '"': '\\"',
                "\\'": "'",
                '\\\n': '',
                '\\x': '\\u00',
            }.get(m.group(0), m.group(0)), v[1:-1])

        for regex, base in INTEGER_TABLE:
            im = re.match(regex, v)
            if im:
                i = int(im.group(1), base)
                return '"%d":' % i if v.endswith(':') else '%d' % i

        return '"%s"' % v

    return re.sub(r'''(?sx)
        "(?:[^"\\]*(?:\\\\|\\['"nurtbfx/\n]))*[^"\\]*"|
        '(?:[^'\\]*(?:\\\\|\\['"nurtbfx/\n]))*[^'\\]*'|
        {comment}|,(?={skip}[\]}}])|
        [a-zA-Z_][.a-zA-Z_0-9]*|
        \b(?:0[xX][0-9a-fA-F]+|0+[0-7]+)(?:{skip}:)?|
        [0-9]+(?={skip}:)
        '''.format(comment=COMMENT_RE, skip=SKIP_RE), fix_kv, code)


# _extract.py
# -------------------------------------------------------------------------------------
class Udemy(ProgressBar):

    def __init__(self):
        self._session = ''
        self._cookies = ''

    def _clean(self, text):
        ok = re.compile(r'[^\\/:*?"<>|]')
        text = "".join(x if ok.match(x) else "_" for x in text)
        text = re.sub(r'\.+$', '', text.strip())
        return text

    def _course_name(self, url):
        mobj = re.search(r'(?i)(?://(?P<portal_name>.+?).udemy.com/(?:course(/draft)*/)?(?P<name_or_id>[a-zA-Z0-9_-]+))', url)
        if mobj:
            return mobj.group('portal_name'), mobj.group('name_or_id')

    def _extract_cookie_string(self, raw_cookies):
        cookies = {}
        try:
            # client_id = re.search(r'(?i)(?:client_id=(?P<client_id>\w+))', raw_cookies)
            access_token = re.search(r'(?i)(?:access_token=(?P<access_token>\w+))', raw_cookies)
        except:
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Cookies error, Request Headers is required.\n")
            sys.stdout.write(
                fc + sd + "[" + fm + sb + "i" + fc + sd + "] : " + fg + sb + "Copy Request Headers for single request to a file, while you are logged in.\n")
            sys.exit(0)
        if not access_token:
            sys.stdout.write(
                fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Cookies error, unable to find access_token, proper cookies required.\n")
            sys.stdout.flush()
            sys.exit(0)
        access_token = access_token.group('access_token')
        cookies.update({'access_token': access_token})
        # 'client_id': client_id.group('client_id'),
        return cookies

    def _sanitize(self, unsafetext):
        text = sanitize(slugify(unsafetext, lower=False, spaces=True, ok=SLUG_OK + '().[]'))
        return text

    def _login(self, username='', password='', cookies=''):
        if not cookies:
            auth = UdemyAuth(username=username, password=password)
            self._session = auth.authenticate()
        if cookies:
            self._cookies = self._extract_cookie_string(raw_cookies=cookies)
            access_token = self._cookies.get('access_token')
            client_id = self._cookies.get('client_id')
            time.sleep(0.3)
            auth = UdemyAuth()
            self._session = auth.authenticate(access_token=access_token, client_id=client_id)
            self._session._session.cookies.update(self._cookies)
        if self._session is not None:
            return {'login': 'successful'}
        else:
            return {'login': 'failed'}

    def _logout(self):
        return self._session.terminate()

    def _subscribed_courses(self, portal_name, course_name):
        results = []
        self._session._headers.update({
            'Host': '{portal_name}.udemy.com'.format(portal_name=portal_name),
            'Referer': 'https://{portal_name}.udemy.com/home/my-courses/search/?q={course_name}'.format(portal_name=portal_name,
                                                                                                        course_name=course_name)
        })
        url = COURSE_SEARCH.format(portal_name=portal_name, course_name=course_name)
        try:
            webpage = self._session._get(url).json()
        except conn_error as e:
            sys.stdout.write(
                fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
            time.sleep(0.8)
            sys.exit(0)
        except (ValueError, Exception) as e:
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "%s.\n" % (e))
            time.sleep(0.8)
            sys.exit(0)
        else:
            results = webpage.get('results', [])
        return results

    def _my_courses(self, portal_name):
        results = []
        try:
            url = MY_COURSES_URL.format(portal_name=portal_name)
            webpage = self._session._get(url).json()
        except conn_error as e:
            sys.stdout.write(
                fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
            time.sleep(0.8)
            sys.exit(0)
        except (ValueError, Exception) as e:
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "%s.\n" % (e))
            time.sleep(0.8)
            sys.exit(0)
        else:
            results = webpage.get('results', [])
        return results

    def _archived_courses(self, portal_name):
        results = []
        try:
            url = MY_COURSES_URL.format(portal_name=portal_name) + "&is_archived=true"
            webpage = self._session._get(url).json()
        except conn_error as e:
            sys.stdout.write(
                fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
            time.sleep(0.8)
            sys.exit(0)
        except (ValueError, Exception) as e:
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "%s.\n" % (e))
            time.sleep(0.8)
            sys.exit(0)
        else:
            results = webpage.get('results', [])
        return results

    def _subscribed_collection_courses(self, portal_name):
        url = COLLECTION_URL.format(portal_name=portal_name)
        courses_lists = []
        try:
            webpage = self._session._get(url).json()
        except conn_error as e:
            sys.stdout.write(
                fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
            time.sleep(0.8)
            sys.exit(0)
        except (ValueError, Exception) as e:
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "%s.\n" % (e))
            time.sleep(0.8)
            sys.exit(0)
        else:
            results = webpage.get('results', [])
            if results:
                [courses_lists.extend(courses.get('courses', [])) for courses in results if courses.get('courses', [])]
        return courses_lists

    def __extract_course(self, response, course_name):
        _temp = {}
        if response:
            for entry in response:
                course_id = str(entry.get('id'))
                published_title = entry.get('published_title')
                if course_name in (published_title, course_id):
                    _temp = entry
                    break
        return _temp

    def _extract_course_info(self, url):
        portal_name, course_name = self._course_name(url)
        course = {}
        results = self._subscribed_courses(portal_name=portal_name, course_name=course_name)
        course = self.__extract_course(response=results, course_name=course_name)
        if not course:
            results = self._my_courses(portal_name=portal_name)
            course = self.__extract_course(response=results, course_name=course_name)
        if not course:
            results = self._subscribed_collection_courses(portal_name=portal_name)
            course = self.__extract_course(response=results, course_name=course_name)
        if not course:
            results = self._archived_courses(portal_name=portal_name)
            course = self.__extract_course(response=results, course_name=course_name)

        if course:
            course.update({'portal_name': portal_name})
            return course.get('id'), course
        if not course:
            sys.stdout.write(
                '\033[2K\033[1G\r\r' + fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fg + sb + "Downloading course information, course id not found .. (%s%sfailed%s%s)\n" % (
                fr, sb, fg, sb))
            sys.stdout.write(
                fc + sd + "[" + fw + sb + "i" + fc + sd + "] : " + fw + sb + "It seems either you are not enrolled or you have to visit the course atleast once while you are logged in.\n")
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to logout now...\n")
            if not self._cookies:
                self._logout()
            sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sb + "Logged out successfully.\n")
            sys.exit(0)

    def _extract_large_course_content(self, url):
        url = url.replace('10000', '300') if url.endswith('10000') else url
        try:
            data = self._session._get(url).json()
        except conn_error as e:
            sys.stdout.write(
                fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
            time.sleep(0.8)
            sys.exit(0)
        else:
            _next = data.get('next')
            while _next:
                try:
                    resp = self._session._get(_next).json()
                except conn_error as e:
                    sys.stdout.write(
                        fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
                    time.sleep(0.8)
                    sys.exit(0)
                else:
                    _next = resp.get('next')
                    results = resp.get('results')
                    if results and isinstance(results, list):
                        for d in resp['results']:
                            data['results'].append(d)
            return data

    def _extract_course_json(self, url, course_id, portal_name):
        self._session._headers.update({'Referer': url})
        url = COURSE_URL.format(portal_name=portal_name, course_id=course_id)
        try:
            resp = self._session._get(url)
            if resp.status_code in [502, 503]:
                resp = self._extract_large_course_content(url=url)
            else:
                resp = resp.json()
        except conn_error as e:
            sys.stdout.write(
                fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Connection error : make sure your internet connection is working.\n")
            time.sleep(0.8)
            sys.exit(0)
        except (ValueError, Exception) as e:
            resp = self._extract_large_course_content(url=url)
            return resp
        else:
            return resp

    def _html_to_json(self, view_html, lecture_id):
        data = parse_json(
            search_regex(
                r'videojs-setup-data=(["\'])(?P<data>{.+?})\1',
                view_html,
                'setup data',
                default='{}',
                group='data'),
            lecture_id,
            transform_source=unescapeHTML,
            fatal=False
        )
        text_tracks = parse_json(
            search_regex(
                r'text-tracks=(["\'])(?P<data>\[.+?\])\1',
                view_html,
                'text tracks',
                default='{}',
                group='data'),
            lecture_id,
            transform_source=lambda s: js_to_json(unescapeHTML(s)),
            fatal=False
        )
        return data, text_tracks

    def _extract_ppt(self, assets):
        _temp = []
        download_urls = assets.get('download_urls')
        slides_urls = assets.get('slide_urls')
        filename = self._sanitize(assets.get('filename'))
        if download_urls and isinstance(download_urls, dict):
            extension = filename.rsplit('.', 1)[-1] if '.' in filename else ''
            download_url = download_urls.get('Presentation', [])[0].get('file')
            _temp.append({
                'type': 'presentation',
                'filename': filename,
                'extension': extension,
                'download_url': download_url
            })
        return _temp

    def _extract_file(self, assets):
        _temp = []
        download_urls = assets.get('download_urls')
        filename = self._sanitize(assets.get('filename'))
        if download_urls and isinstance(download_urls, dict):
            extension = filename.rsplit('.', 1)[-1] if '.' in filename else ''
            download_url = download_urls.get('File', [])[0].get('file')
            _temp.append({
                'type': 'file',
                'filename': filename,
                'extension': extension,
                'download_url': download_url
            })
        return _temp

    def _extract_ebook(self, assets):
        _temp = []
        download_urls = assets.get('download_urls')
        filename = self._sanitize(assets.get('filename'))
        if download_urls and isinstance(download_urls, dict):
            extension = filename.rsplit('.', 1)[-1] if '.' in filename else ''
            download_url = download_urls.get('E-Book', [])[0].get('file')
            _temp.append({
                'type': 'ebook',
                'filename': filename,
                'extension': extension,
                'download_url': download_url
            })
        return _temp

    def _extract_audio(self, assets):
        _temp = []
        download_urls = assets.get('download_urls')
        filename = self._sanitize(assets.get('filename'))
        if download_urls and isinstance(download_urls, dict):
            extension = filename.rsplit('.', 1)[-1] if '.' in filename else ''
            download_url = download_urls.get('Audio', [])[0].get('file')
            _temp.append({
                'type': 'audio',
                'filename': filename,
                'extension': extension,
                'download_url': download_url
            })
        return _temp

    def _extract_sources(self, sources):
        _temp = []
        if sources and isinstance(sources, list):
            for source in sources:
                label = source.get('label')
                download_url = source.get('file')
                if not download_url:
                    continue
                if label.lower() == 'audio':
                    continue
                height = label if label else None
                if height == "2160":
                    width = "3840"
                elif height == "1440":
                    width = "2560"
                elif height == "1080":
                    width = "1920"
                elif height == "720":
                    width = "1280"
                elif height == "480":
                    width = "854"
                elif height == "360":
                    width = "640"
                elif height == "240":
                    width = "426"
                else:
                    width = "256"
                if source.get('type') == 'application/x-mpegURL' or 'm3u8' in download_url:
                    continue
                else:
                    _type = source.get('type')
                    _temp.append({
                        'type': 'video',
                        'height': height,
                        'width': width,
                        'extension': _type.replace('video/', ''),
                        'download_url': download_url,
                    })
        return _temp

    def _extract_subtitles(self, tracks):
        _temp = []
        if tracks and isinstance(tracks, list):
            for track in tracks:
                if not isinstance(track, dict):
                    continue
                if track.get('_class') != 'caption':
                    continue
                download_url = track.get('url')
                if not download_url or not isinstance(download_url, encoding):
                    continue
                lang = track.get('language') or track.get('srclang') or track.get('label') or track['locale_id'].split('_')[0]
                ext = 'vtt' if 'vtt' in download_url.rsplit('.', 1)[-1] else 'srt'
                _temp.append({
                    'type': 'subtitle',
                    'language': lang,
                    'extension': ext,
                    'download_url': download_url,
                })
        return _temp

    def _extract_supplementary_assets(self, supp_assets):
        _temp = []
        for entry in supp_assets:
            file_id = entry.get('id')
            filename = self._sanitize(entry.get('filename'))
            download_urls = entry.get('download_urls')
            external_url = entry.get('external_url')
            slide_url = entry.get('slide_urls')
            asset_type = entry.get('asset_type').lower()
            if asset_type == 'file':
                if download_urls and isinstance(download_urls, dict):
                    extension = filename.rsplit('.', 1)[-1] if '.' in filename else ''
                    download_url = download_urls.get('File', [])[0].get('file')
                    _temp.append({
                        'type': 'file',
                        'filename': filename,
                        'extension': extension,
                        'download_url': download_url,
                    })
            elif asset_type == 'sourcecode':
                if download_urls and isinstance(download_urls, dict):
                    extension = filename.rsplit('.', 1)[-1] if '.' in filename else ''
                    download_url = download_urls.get('SourceCode', [])[0].get('file')
                    _temp.append({
                        'type': 'source_code',
                        'filename': filename,
                        'extension': extension,
                        'download_url': download_url,
                    })
            elif asset_type == 'externallink':
                _temp.append({
                    'type': 'external_link',
                    'filename': filename,
                    'extension': 'txt',
                    'download_url': external_url,
                })
        return _temp

    def _real_extract(self, url=''):

        _udemy = {}
        course_id, course_info = self._extract_course_info(url)

        if course_info and isinstance(course_info, dict):
            course_title = course_info.get('published_title')
            portal_name = course_info.get('portal_name')

        course_json = self._extract_course_json(url, course_id, portal_name)
        course = course_json.get('results')
        resource = course_json.get('detail')

        if resource:
            if not self._cookies:
                sys.stdout.write(
                    fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Udemy Says : {}{}{} Run udemy-dl against course within few seconds.\n".format(
                        resource, fw, sb))
            if self._cookies:
                sys.stdout.write(
                    fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Udemy Says : {}{}{} cookies seems to be expired.\n".format(resource,
                                                                                                                                             fw, sb))
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to logout now...\n")
            if not self._cookies:
                self._logout()
            sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sb + "Logged out successfully.\n")
            sys.exit(0)

        _udemy['course_id'] = course_id
        _udemy['course_title'] = course_title
        _udemy['chapters'] = []

        counter = -1

        if course:
            for entry in course:

                clazz = entry.get('_class')
                asset = entry.get('asset')
                supp_assets = entry.get('supplementary_assets')

                if clazz == 'chapter':
                    lectures = []
                    chapter_index = entry.get('object_index')
                    chapter_title = self._clean(self._sanitize(entry.get('title')))
                    chapter = "{0:02d} {1!s}".format(chapter_index, chapter_title)
                    unsafe_chapter = u'{0:02d} '.format(chapter_index) + self._clean(entry.get('title'))
                    if chapter not in _udemy['chapters']:
                        _udemy['chapters'].append({
                            'chapter_title': chapter,
                            'chapter_id': entry.get("id"),
                            'chapter_index': chapter_index,
                            'unsafe_chapter': unsafe_chapter,
                            'lectures': [],
                        })
                        counter += 1
                elif clazz == 'lecture':

                    lecture_id = entry.get("id")
                    if len(_udemy['chapters']) == 0:
                        lectures = []
                        chapter_index = entry.get('object_index')
                        chapter_title = self._clean(self._sanitize(entry.get('title')))
                        chapter = "{0:03d} {1!s}".format(chapter_index, chapter_title)
                        unsafe_chapter = u'{0:02d} '.format(chapter_index) + self._clean(entry.get('title'))
                        if chapter not in _udemy['chapters']:
                            _udemy['chapters'].append({
                                'chapter_title': chapter,
                                'chapter_id': lecture_id,
                                'chapter_index': chapter_index,
                                'unsafe_chapter': unsafe_chapter,
                                'lectures': [],
                            })
                            counter += 1

                    if lecture_id:

                        view_html = entry.get('view_html')
                        retVal = []

                        if isinstance(asset, dict):
                            asset_type = asset.get('asset_type').lower() or asset.get('assetType').lower()
                            if asset_type == 'article':
                                if isinstance(supp_assets, list) and len(supp_assets) > 0:
                                    retVal = self._extract_supplementary_assets(supp_assets)
                            elif asset_type == 'video':
                                if isinstance(supp_assets, list) and len(supp_assets) > 0:
                                    retVal = self._extract_supplementary_assets(supp_assets)
                            elif asset_type == 'e-book':
                                retVal = self._extract_ebook(asset)
                            elif asset_type == 'file':
                                retVal = self._extract_file(asset)
                            elif asset_type == 'presentation':
                                retVal = self._extract_ppt(asset)
                            elif asset_type == 'audio':
                                retVal = self._extract_audio(asset)

                        if view_html:
                            text = '\r' + fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Downloading course information .. "
                            self._spinner(text)
                            lecture_index = entry.get('object_index')
                            lecture_title = self._clean(self._sanitize(entry.get('title')))
                            lecture = "{0:03d} {1!s}".format(lecture_index, lecture_title)
                            unsafe_lecture = u'{0:03d} '.format(lecture_index) + entry.get('title')
                            data, subs = self._html_to_json(view_html, lecture_id)
                            if data and isinstance(data, dict):
                                sources = data.get('sources')
                                tracks = data.get('tracks') if isinstance(data.get('tracks'), list) else subs
                                duration = data.get('duration')
                                lectures.append({
                                    'lecture_index': lecture_index,
                                    'lectures_id': lecture_id,
                                    'lecture_title': lecture,
                                    'unsafe_lecture': unsafe_lecture,
                                    'duration': duration,
                                    'assets': retVal,
                                    'assets_count': len(retVal),
                                    'sources': self._extract_sources(sources),
                                    'subtitles': self._extract_subtitles(tracks),
                                    'subtitle_count': len(self._extract_subtitles(tracks)),
                                    'sources_count': len(self._extract_sources(sources)),
                                })
                            else:
                                lectures.append({
                                    'lecture_index': lecture_index,
                                    'lectures_id': lecture_id,
                                    'lecture_title': lecture,
                                    'unsafe_lecture': unsafe_lecture,
                                    'html_content': view_html,
                                    'extension': 'html',
                                    'assets': retVal,
                                    'assets_count': len(retVal),
                                    'subtitle_count': 0,
                                    'sources_count': 0,
                                })
                        if not view_html:
                            text = '\r' + fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Downloading course information .. "
                            self._spinner(text)
                            lecture_index = entry.get('object_index')
                            lecture_title = self._clean(self._sanitize(entry.get('title')))
                            lecture = "{0:03d} {1!s}".format(lecture_index, lecture_title)
                            unsafe_lecture = u'{0:03d} '.format(lecture_index) + self._clean(entry.get('title'))
                            data = asset.get('stream_urls')
                            if data and isinstance(data, dict):
                                sources = data.get('Video')
                                tracks = asset.get('captions')
                                duration = asset.get('time_estimation')
                                lectures.append({
                                    'lecture_index': lecture_index,
                                    'lectures_id': lecture_id,
                                    'lecture_title': lecture,
                                    'unsafe_lecture': unsafe_lecture,
                                    'duration': duration,
                                    'assets': retVal,
                                    'assets_count': len(retVal),
                                    'sources': self._extract_sources(sources),
                                    'subtitles': self._extract_subtitles(tracks),
                                    'subtitle_count': len(self._extract_subtitles(tracks)),
                                    'sources_count': len(self._extract_sources(sources)),
                                })
                            else:
                                lectures.append({
                                    'lecture_index': lecture_index,
                                    'lectures_id': lecture_id,
                                    'lecture_title': lecture,
                                    'unsafe_lecture': unsafe_lecture,
                                    'html_content': asset.get('body'),
                                    'extension': 'html',
                                    'assets': retVal,
                                    'assets_count': len(retVal),
                                    'subtitle_count': 0,
                                    'sources_count': 0,
                                })

                    _udemy['chapters'][counter]['lectures'] = lectures
                    _udemy['chapters'][counter]['lectures_count'] = len(lectures)
                elif clazz == 'quiz':
                    lecture_id = entry.get("id")
                    if len(_udemy['chapters']) == 0:
                        lectures = []
                        chapter_index = entry.get('object_index')
                        chapter_title = self._clean(self._sanitize(entry.get('title')))
                        chapter = "{0:03d} {1!s}".format(chapter_index, chapter_title)
                        unsafe_chapter = u'{0:02d} '.format(chapter_index) + self._clean(entry.get('title'))
                        if chapter not in _udemy['chapters']:
                            _udemy['chapters'].append({
                                'chapter_title': chapter,
                                'unsafe_chapter': unsafe_chapter,
                                'chapter_id': lecture_id,
                                'chapter_index': chapter_index,
                                'lectures': [],
                            })
                            counter += 1
                    _udemy['chapters'][counter]['lectures'] = lectures
                    _udemy['chapters'][counter]['lectures_count'] = len(lectures)
            _udemy['total_chapters'] = len(_udemy['chapters'])
            _udemy['total_lectures'] = sum([entry.get('lectures_count', 0) for entry in _udemy['chapters'] if entry])

        return _udemy


# _shared.py
# -------------------------------------------------------------------------------------
class Downloader(object):

    def __init__(self):
        self._url = None
        self._filename = None
        self._mediatype = None
        self._extension = None
        self._sess = requests.session()

    @property
    def url(self):
        """abac"""
        return self._url

    @property
    def mediatype(self):
        return self._mediatype

    @property
    def extension(self):
        return self._extension

    @property
    def filename(self):
        if not self._filename:
            self._filename = self._generate_filename()
        return self._filename

    @property
    def unsafe_filename(self):
        if not self._filename:
            self._filename = self._generate_unsafe_filename()
        return self._filename

    def _generate_filename():
        pass

    def _generate_unsafe_filename():
        pass

    def _write_external_links(self, filepath, unsafe=False):
        retVal = {}
        savedirs, name = os.path.split(filepath)
        filename = 'external-assets-links.txt' if not unsafe else u'external-assets-links.txt'
        filename = os.path.join(savedirs, filename)

        file_data = []
        if os.path.isfile(filename):
            file_data = [i.strip().lower() for i in open(filename) if i]

        try:
            f = codecs.open(filename, 'a', encoding='utf-8', errors='ignore')
            data = '\n{}\n{}\n'.format(name, self.url) if not unsafe else u'\n{}\n{}\n'.format(
                name, self.url)
            if name.lower() not in file_data:
                f.write(data)
        except (OSError, Exception, UnicodeDecodeError) as e:
            retVal = {'status' : 'False', 'msg' : '{}'.format(e)}
        else:
            retVal = {'status' : 'True', 'msg' : 'download'}
            f.close()

        return retVal

    def download(self, filepath="", unsafe=False, quiet=False, callback=lambda *x: None):
        savedir = filename = ""
        retVal = {}

        if filepath and os.path.isdir(filepath):
            savedir, filename = filepath, self.filename if not unsafe else self.unsafe_filename

        elif filepath:
            savedir, filename = os.path.split(filepath)

        else:
            filename = self.filename if not unsafe else self.unsafe_filename

        filepath = os.path.join(savedir, filename)
        if os.name == "nt" and len(filepath) > 250:
            filepath = "\\\\?\\{}".format(filepath)

        if self.mediatype == 'external_link':
            return self._write_external_links(filepath, unsafe)

        if filepath and filepath.endswith('.vtt'):
            filepath_vtt2srt = filepath.replace('.vtt', '.srt')
            if os.path.isfile(filepath_vtt2srt):
                retVal = {"status" : "True", "msg" : "already downloaded"}
                return retVal

        if os.path.isfile(filepath):
            retVal = {"status": "True", "msg": "already downloaded"}
            return retVal

        temp_filepath = filepath + ".part"

        self._active = True
        bytes_to_be_downloaded = 0
        fmode, offset = "wb", 0
        chunksize, bytesdone, t0 = 16384, 0, time.time()
        headers = {'User-Agent': HEADERS.get('User-Agent')}
        if os.path.exists(temp_filepath):
            offset = os.stat(temp_filepath).st_size

        if offset:
            offset_range = 'bytes={}-'.format(offset)
            headers['Range'] = offset_range
            bytesdone = offset
            fmode = "ab"

        status_string = ('  {:,} Bytes [{:.2%}] received. Rate: [{:4.0f} '
                         'KB/s].  ETA: [{:.0f} secs]')

        try:
            try:
                response = self._sess.get(self.url, headers=headers, stream=True, timeout=10)
            except conn_error as error:
                return {'status': 'False', 'msg': 'ConnectionError: %s' % (str(error))}
            if response.ok:
                bytes_to_be_downloaded = total = int(response.headers.get('Content-Length'))
                if bytesdone > 0:
                    bytes_to_be_downloaded = bytes_to_be_downloaded + bytesdone
                total = bytes_to_be_downloaded
                with open(temp_filepath, fmode) as media_file:
                    is_malformed = False
                    for chunk in response.iter_content(chunksize):
                        if not chunk:
                            break
                        media_file.write(chunk)
                        elapsed = time.time() - t0
                        bytesdone += len(chunk)
                        if elapsed:
                            try:
                                rate = ((float(bytesdone) - float(offset)) / 1024.0) / elapsed
                                eta = (total - bytesdone) / (rate * 1024.0)
                            except ZeroDivisionError:
                                is_malformed = True
                                try:
                                    os.unlink(temp_filepath)
                                except Exception:
                                    pass
                                retVal = {"status" : "False", "msg" : "ZeroDivisionError : it seems, lecture has malfunction or is zero byte(s) .."}
                                break
                        else:
                            rate = 0
                            eta = 0

                        if not is_malformed:
                            progress_stats = (
                                bytesdone, bytesdone * 1.0 / total, rate, eta)

                            if not quiet:
                                status = status_string.format(*progress_stats)
                                sys.stdout.write(
                                    "\r" + status + ' ' * 4 + "\r")
                                sys.stdout.flush()

                            if callback:
                                callback(total, *progress_stats)
            if not response.ok:
                code = response.status_code
                reason = response.reason
                retVal = {
                    "status": "False", "msg": "Udemy returned HTTP Code %s: %s" % (code, reason)}
                response.close()
        except KeyboardInterrupt as error:
            raise error
        except Exception as error:
            retVal = {"status": "False",
                      "msg": "Reason : {}".format(str(error))}
            return retVal
        # check if file is downloaded completely
        if os.path.isfile(temp_filepath):
            total_bytes_done = os.stat(temp_filepath).st_size
            if total_bytes_done == bytes_to_be_downloaded:
                self._active = False
            if total_bytes_done < bytes_to_be_downloaded:
                # set active to be True as remaining bytes to be downloaded
                self._active = True
                # try downloading back again remaining bytes until we download completely
                self.download(filepath=filepath,
                              unsafe=unsafe,
                              quiet=quiet)


        if not self._active:
            os.rename(temp_filepath, filepath)
            retVal = {"status": "True", "msg": "download"}

        return retVal

class UdemyCourse(object):

    def __init__(self, url, username='', password='', cookies='', basic=True, callback=None):

        self._url = url
        self._username = username
        self._password = password
        self._cookies = cookies 
        self._callback = callback or (lambda x: None)
        self._have_basic = False

        self._id = None
        self._title = None
        self._chapters_count = None
        self._total_lectures = None

        self._chapters = []

        if basic:
            self._fetch_course()

    def _fetch_course(self):
        raise NotImplementedError

    @property
    def id(self):
        if not self._id:
            self._fetch_course()
        return self._id

    @property
    def title(self):
        if not self._title:
            self._fetch_course()
        return self._title

    @property
    def chapters(self):
        if not self._chapters_count:
            self._fetch_course()
        return self._chapters_count

    @property
    def lectures(self):
        if not self._total_lectures:
            self._fetch_course()
        return self._total_lectures

    def get_chapters(self):
        if not self._chapters:
            self._fetch_course()
        return self._chapters

class UdemyChapters(object):

    def __init__(self):

        self._chapter_id = None
        self._chapter_index = None
        self._chapter_title = None
        self._unsafe_title = None
        self._lectures_count = None

        self._lectures = []

    def __repr__(self):
        chapter = "{title}".format(title=self.title)
        return chapter

    @property
    def id(self):
        return self._chapter_id

    @property
    def index(self):
        return self._chapter_index

    @property
    def title(self):
        return self._chapter_title

    @property
    def unsafe_title(self):
        return self._unsafe_title
    
    @property
    def lectures(self):
        return self._lectures_count

    def get_lectures(self):
        return self._lectures

class UdemyLectures(object):

    def __init__(self):

        self._best = None
        self._duration = None
        self._extension = None
        self._lecture_id = None
        self._lecture_title  =   None
        self._unsafe_title = None
        self._lecture_index = None
        self._sources_count = None
        self._assets_count = None
        self._subtitles_count = None
        self._html_content = None

        self._assets = []
        self._streams = []
        self._subtitles = []

    def __repr__(self):
        lecture = "{title}".format(title=self.title)
        return lecture
    
    @property
    def id(self):
        return self._lecture_id

    @property
    def index(self):
        return self._lecture_index

    @property
    def title(self):
        return self._lecture_title

    @property
    def unsafe_title(self):
        return self._unsafe_title

    @property
    def html(self):
        return self._html_content
    
    @property
    def duration(self):
        return self._duration

    @property
    def extension(self):
        return self._extension

    @property
    def assets(self):
        if not self._assets:
            self._process_assets()
        return self._assets

    @property
    def streams(self):
        if not self._streams:
            self._process_streams()
        return self._streams

    @property
    def subtitles(self):
        if not self._subtitles:
            self._process_subtitles()
        return self._subtitles

    def _getbest(self):
        streams = self.streams
        if not streams:
            return None
        def _sortkey(x, keyres=0, keyftype=0):
            keyres = int(x.resolution.split('x')[0])
            keyftype = x.extension
            st = (keyftype, keyres)
            return st
        
        self._best = max(streams, key=_sortkey)
        return self._best

    def getbest(self):
        return self._getbest()

    def dump(self, filepath, unsafe=False):
        retVal = {}
        data = '''
                <html>
                <head>
                <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">
                <title>%s</title>
                </head>
                <body>
                <div class="container">
                <div class="row">
                <div class="col-md-10 col-md-offset-1">
                    <p class="lead">%s</p>
                </div>
                </div>
                </div>
                <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js" integrity="sha384-Tc5IQib027qvyjSMfHjOMaLkfuWVxZxUPnCJA7l2mCWNIpG9mGCD8wGNIcPD7Txa" crossorigin="anonymous"></script>
                </body>
                </html>
                ''' % (self.title, self.html)
        html = data.encode('utf-8').strip()
        if not unsafe:
            filename = "%s\\%s" % (filepath, self.title) if os.name == 'nt' else "%s/%s" % (filepath, self.title)
            filename += ".html"
        if unsafe:
            filename = u"%s\\%s" % (filepath, self.unsafe_title) if os.name == 'nt' else u"%s/%s" % (filepath, self.unsafe_title)
            filename += ".html"

        if os.path.isfile(filename):
            retVal = {"status" : "True", "msg" : "already downloaded"}
            return retVal
        
        try:
            f = codecs.open(filename, 'wb', errors='ignore')
            f.write(html)
        except (OSError, Exception, UnicodeDecodeError) as e:
            retVal = {'status' : 'False', 'msg' : '{}'.format(e)}
        else:
            retVal = {'status' : 'True', 'msg' : 'download'}
            f.close()

        return retVal

class UdemyLectureStream(Downloader):


    def __init__(self, parent):

        self._mediatype = None
        self._quality = None
        self._resolution = None
        self._dimention = None
        self._extension = None
        self._url = None

        self._parent = parent
        self._filename = None
        self._fsize = None
        self._active = False

        Downloader.__init__(self)

    def __repr__(self):
        out = "%s:%s@%s" % (self.mediatype, self.extension, self.quality)
        return out

    def _generate_filename(self):
        ok = re.compile(r'[^\\/:*?"<>|]')
        filename = "".join(x if ok.match(x) else "_" for x in self.title)
        filename += "." + self.extension
        return filename

    def _generate_unsafe_filename(self):
        ok = re.compile(r'[^\\/:*?"<>|]')
        filename = "".join(x if ok.match(x) else "_" for x in self.unsafe_title)
        filename += "." + self.extension
        return filename

    @property
    def resolution(self):
        return self._resolution

    @property
    def quality(self):
        return self._quality

    @property
    def url(self):
        return self._url

    @property
    def id(self):
        return self._parent.id

    @property
    def dimention(self):
        return self._dimention

    @property
    def extension(self):
        return self._extension

    @property
    def filename(self):
        if not self._filename:
            self._filename = self._generate_filename()
        return self._filename

    @property
    def title(self):
        return self._parent.title

    @property
    def unsafe_title(self):
        return self._parent.unsafe_title

    @property
    def unsafe_filename(self):
        if not self._filename:
            self._filename = self._generate_unsafe_filename()
        return self._filename

    @property
    def mediatype(self):
        return self._mediatype

    def get_filesize(self):
        if not self._fsize:
            headers = {'User-Agent': HEADERS.get('User-Agent')}
            try:
                with requests.get(self.url, stream=True, headers=headers) as resp:
                    if resp.ok:
                        self._fsize = float(resp.headers.get('Content-Length', 0))
                    if not resp.ok:
                        self._fsize = 0
            except conn_error:
                self._fsize = 0
        return self._fsize

class UdemyLectureAssets(Downloader):

    def __init__(self, parent):

        self._extension = None
        self._mediatype = None
        self._url = None

        self._parent = parent
        self._filename = None
        self._fsize = None
        self._active = False

        Downloader.__init__(self)

    def __repr__(self):
        out = "%s:%s@%s" % (self.mediatype, self.extension, self.extension)
        return out

    def _generate_filename(self):
        ok = re.compile(r'[^\\/:*?"<>|]')
        filename = "".join(x if ok.match(x) else "_" for x in self.title)
        filename += ".{}".format(self.extension)
        return filename

    def _generate_unsafe_filename(self):
        ok = re.compile(r'[^\\/:*?"<>|]')
        filename = "".join(x if ok.match(x) else "_" for x in self.unsafe_title)
        filename += ".{}".format(self.extension)
        return filename

    @property
    def id(self):
        return self._parent.id

    @property
    def url(self):
        return self._url

    @property
    def extension(self):
        return self._extension

    @property
    def title(self):
        return self._parent.title

    @property
    def unsafe_title(self):
        return self._parent.unsafe_title

    @property
    def filename(self):
        if not self._filename:
            self._filename = self._generate_filename()
        return self._filename

    @property
    def unsafe_filename(self):
        if not self._filename:
            self._filename = self._generate_unsafe_filename()
        return self._filename

    @property
    def mediatype(self):
        return self._mediatype

    def get_filesize(self):
        if not self._fsize:
            headers = {'User-Agent': HEADERS.get('User-Agent')}
            try:
                with requests.get(self.url, stream=True, headers=headers) as resp:
                    if resp.ok:
                        self._fsize = float(resp.headers.get('Content-Length', 0))
                    if not resp.ok:
                        self._fsize = 0
            except conn_error:
                self._fsize = 0
        return self._fsize

class UdemyLectureSubtitles(Downloader):

    def __init__(self, parent):

        self._mediatype = None
        self._extension = None
        self._language = None
        self._url = None

        self._parent = parent
        self._filename = None
        self._fsize = None
        self._active = False

        Downloader.__init__(self)

    def __repr__(self):
        out = "%s:%s@%s" % (self.mediatype, self.language, self.extension)
        return out

    def _generate_filename(self):
        ok = re.compile(r'[^\\/:*?"<>|]')
        filename = "".join(x if ok.match(x) else "_" for x in self.title)
        filename += ".{}.{}".format(self.language, self.extension)
        return filename

    def _generate_unsafe_filename(self):
        ok = re.compile(r'[^\\/:*?"<>|]')
        filename = "".join(x if ok.match(x) else "_" for x in self.unsafe_title)
        filename += ".{}.{}".format(self.language, self.extension)
        return filename

    @property
    def id(self):
        return self._parent.id
    
    @property
    def url(self):
        return self._url

    @property
    def extension(self):
        return self._extension

    @property
    def language(self):
        return self._language

    @property
    def title(self):
        return self._parent.title

    @property
    def unsafe_title(self):
        return self._parent.unsafe_title

    @property
    def filename(self):
        if not self._filename:
            self._filename = self._generate_filename()
        return self._filename

    @property
    def unsafe_filename(self):
        if not self._filename:
            self._filename = self._generate_unsafe_filename()
        return self._filename

    @property
    def mediatype(self):
        return self._mediatype

    def get_filesize(self):
        if not self._fsize:
            headers = {'User-Agent': HEADERS.get('User-Agent')}
            try:
                with requests.get(self.url, stream=True, headers=headers) as resp:
                    if resp.ok:
                        self._fsize = float(resp.headers.get('Content-Length', 0))
                    if not resp.ok:
                        self._fsize = 0
            except conn_error:
                self._fsize = 0
        return self._fsize


# _internal.py
# -------------------------------------------------------------------------------------
class InternUdemyCourse(UdemyCourse, Udemy):
    def __init__(self, *args, **kwargs):
        self._info    = ''
        super(InternUdemyCourse, self).__init__(*args, **kwargs)

    def _fetch_course(self):
        if self._have_basic:
            return
        if not self._cookies:
            auth = self._login(username=self._username, password=self._password)
        if self._cookies:
            auth = self._login(cookies=self._cookies)
        if auth.get('login') == 'successful':
            sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sb + "Logged in successfully.\n")
            sys.stdout.write('\r' + fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Downloading course information .. \r")
            self._info              =       self._real_extract(self._url)
            time.sleep(1)
            sys.stdout.write('\r' + fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Downloaded course information .. (done)\r\n")
            self._id                =       self._info['course_id']
            self._title             =       self._info['course_title']
            self._chapters_count    =       self._info['total_chapters']
            self._total_lectures    =       self._info['total_lectures']
            self._chapters          =       [InternUdemyChapter(z) for z in self._info['chapters']]
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to logout now...\n")
            if not self._cookies:
                self._logout()
            sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sb + "Logged out successfully.\n")
            self._have_basic = True
        if auth.get('login') == 'failed':
            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Failed to login ..\n")
            sys.exit(0)

class InternUdemyChapter(UdemyChapters):
    
    def __init__(self, chapter):
        super(InternUdemyChapter, self).__init__()

        self._chapter_id        = chapter['chapter_id']
        self._chapter_title     = chapter['chapter_title']
        self._unsafe_title      = chapter['unsafe_chapter']
        self._chapter_index     = chapter['chapter_index']
        self._lectures_count    = chapter.get('lectures_count', 0)
        self._lectures          = [InternUdemyLecture(z) for z in chapter['lectures']] if self._lectures_count > 0 else []

class InternUdemyLecture(UdemyLectures):

    def __init__(self, lectures):
        super(InternUdemyLecture, self).__init__()
        self._info              = lectures

        self._lecture_id        = self._info['lectures_id']
        self._lecture_title     = self._info['lecture_title']
        self._unsafe_title      = self._info['unsafe_lecture']
        self._lecture_index     = self._info['lecture_index']
        
        self._subtitles_count   = self._info.get('subtitle_count', 0)
        self._sources_count     = self._info.get('sources_count', 0)
        self._assets_count      = self._info.get('assets_count', 0)
        self._extension         = self._info.get('extension')
        self._html_content      = self._info.get('html_content')
        self._duration          = self._info.get('duration')
        if self._duration:
            duration = int(self._duration)
            (mins, secs) = divmod(duration, 60)
            (hours, mins) = divmod(mins, 60)
            if hours == 0:
                self._duration = "%02d:%02d" % (mins, secs)
            else:
                self._duration = "%02d:%02d:%02d" % (hours, mins, secs)


    def _process_streams(self):
        streams = [InternUdemyLectureStream(z, self) for z in self._info['sources']] if self._sources_count > 0 else []
        self._streams = streams

    def _process_assets(self):
        assets  =   [InternUdemyLectureAssets(z, self) for z in self._info['assets']] if self._assets_count > 0 else []
        self._assets = assets

    def _process_subtitles(self):
        subtitles = [InternUdemyLectureSubtitles(z, self) for z in self._info['subtitles']] if self._subtitles_count > 0 else []
        self._subtitles = subtitles
        
class InternUdemyLectureStream(UdemyLectureStream):

    def __init__(self, sources, parent):
        super(InternUdemyLectureStream, self).__init__(parent)

        self._mediatype = sources.get('type')
        self._extension = sources.get('extension')
        height = sources.get('height', 0)
        width = sources.get('width', 0)
        self._resolution = '%sx%s' % (width, height)
        self._dimention = width, height
        self._quality = self._resolution
        self._url = sources.get('download_url')

class InternUdemyLectureAssets(UdemyLectureAssets):

    def __init__(self, assets, parent):
        super(InternUdemyLectureAssets, self).__init__(parent)

        self._mediatype = assets.get('type')
        self._extension = assets.get('extension')
        self._filename = '{0:03d} {1!s}'.format(parent._lecture_index, assets.get('filename'))
        self._url = assets.get('download_url')

class InternUdemyLectureSubtitles(UdemyLectureSubtitles):

    def __init__(self, subtitles, parent):
        super(InternUdemyLectureSubtitles, self).__init__(parent)

        self._mediatype = subtitles.get('type')
        self._extension = subtitles.get('extension')
        self._language = subtitles.get('language')
        self._url = subtitles.get('download_url')


# _vtt2srt.py
# -------------------------------------------------------------------------------------
class WebVtt2Srt(object):

    _TIMECODE_REGEX = r'(?i)(?P<appeartime>(?:(?:\d{1,2}:)){1,2}\d{2}[\.,]\d+)'
    _TIMECODE = r'(?i)(?P<appeartime>(?:(?:\d{1,2}:)){1,2}\d{2}[\.,]\d+)\s*-->\s*(?i)(?P<disappertime>(?:(?:\d{1,2}:)){1,2}\d{2}[\.,]\d+)'

    def _vttcontents(self, fname):
        try:
            f = codecs.open(filename=fname, encoding='utf-8', errors='ignore')
        except Exception as e:
            return {'status' : 'False', 'msg' : 'failed to open file : file not found ..'}
        content = [line for line in (l.strip() for l in f)]
        f.close()
        return content
        

    def _write_srtcontent(self, fname, content):
        with codecs.open(filename=fname, mode='a', encoding='utf-8') as fd:
            fd.write(content)
        fd.close()

    def _locate_timecode(self, content):
        for (loc, line) in enumerate(content):
            match = re.match(self._TIMECODE_REGEX, line, flags=re.U)
            if match:
                return {'status' : True, 'location' : loc}
        return {'status' : False, 'location' : loc}

    def _is_timecode(self, timecode):
        match = re.match(self._TIMECODE_REGEX, timecode, flags=re.U)
        if match:
            return True
        return False

    def _fix_timecode(self, timecode):
        _sdata = len(timecode.split(',')[0])
        if _sdata == 5:
            timecode = u'00:{code}'.format(code=timecode)
        if _sdata == 7:
            timecode = u'0{code}'.format(code=timecode)
        return timecode

    def _generate_timecode(self, sequence, timecode):
        match = re.match(self._TIMECODE, timecode, flags=re.U)
        if match:
            start, end = self._fix_timecode(timecode=re.sub(r'[\.,]', ',', match.group('appeartime'))), self._fix_timecode(timecode=re.sub(r'[\.,]', ',', match.group('disappertime')))
            return u'{seq}\r\n{appeartime} --> {disappertime}\r\n'.format(seq=sequence, appeartime=start, disappertime=end)
        return u''

    def convert(self, filename=None, remove_vtt=True):
        if filename:
            seq = 1
            fname = filename.replace('.vtt', '.srt')
            content = self._vttcontents(fname=filename)
            if content and isinstance(content, list):
                timecode_loc = self._locate_timecode(content)
                if not timecode_loc.get('status'):
                    return {'status' : 'False', 'msg' : 'subtitle file seems to have malfunction skipping conversion ..'}
                for line in content[timecode_loc.get('location'):]:
                    flag = self._is_timecode(timecode=line)
                    if flag:
                        timecode = self._generate_timecode(seq, unescapeHTML(line))
                        self._write_srtcontent(fname, timecode)
                        seq += 1
                    if not flag:
                        match = re.match('^([0-9]{1,3})$', line, flags=re.U)
                        if not match:
                            data = u'{content}\r\n'.format(content=line)
                            self._write_srtcontent(fname, data)
            else:
                return content
            
            if remove_vtt:
                try:
                    os.unlink(filename)
                except Exception as e:
                    pass
        return {'status' : 'True', 'msg' : 'successfully generated subtitle in srt ...'}


# udemy-dl.py
# -------------------------------------------------------------------------------------
getpass = GetPass()

def get_course(url, username='', password='', cookies='', basic=True, callback=None):
    """Returns udemy course instance.

    @params:
        url      : Udemy course url required : type (string).
        username : Udemy email account required : type (string).
        password : Udemy account password required : type (string)
        cookies  : Udemy account logged in browser cookies optional : type (string)
    """
    return InternUdemyCourse(url, username, password, cookies, basic, callback)

class Udemy(WebVtt2Srt, ProgressBar):

    def __init__(self, url, username='', password='', cookies=''):
        self.url = url
        self.username = username
        self.password = password
        self.cookies = cookies
        super(Udemy, self).__init__()

    def _write_to_file(self, filepath='', lecture='', names_only=False, unsafe=False):
        retVal = {}
        filename = filepath
        filename += '-names-only.txt' if names_only else ".txt"
        fmode = "a"
        if not unsafe:
            title = lecture.title
            url = lecture.url
            url_or_name = url if not names_only else title
            url_or_name += "n"
        if unsafe:
            title = u'%s' % (lecture.unsafe_title)
            url = lecture.url
            url_or_name = url if not names_only else title
            url_or_name += "n"

        try:
            f = codecs.open(filename, fmode, encoding='utf-8', errors='ignore')
            f.write(url_or_name)
        except (OSError, Exception, UnicodeDecodeError) as e:
            retVal = {'status': 'False', 'msg': '{}'.format(e)}
        else:
            retVal = {'status': 'True', 'msg': 'download'}
            f.close()

        return retVal

    def course_save(self, path='', quality='', caption_only=False, skip_captions=False, names_only=False, unsafe=False):
        if not self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login as " + fm + sb + "(%s)" % (
                self.username) + fg + sb + "...n")
        if self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login using cookies ...n")
        course = get_course(url=self.url, username=self.username, password=self.password, cookies=self.cookies)
        course_id = course.id
        course_name = course.title
        total_lectures = course.lectures
        total_chapters = course.chapters
        course_name = (course_name.lower()).replace(' ', '-')
        chapters = course.get_chapters()
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Course " + fb + sb + "'%s'.n" % (course_name))
        sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sd + "Chapter(s) (%s).n" % (total_chapters))
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (total_lectures))
        if path:
            if '~' in path:
                path = os.path.expanduser(path)
            course_path = "%s%s" % (path, course_name) if os.name == 'nt' else "%s/%s" % (path, course_name)
        else:
            path = os.getcwd()
            course_path = "%s%s" % (path, course_name) if os.name == 'nt' else "%s/%s" % (path, course_name)
        filepath = '%s.txt' % (course_path)
        if os.path.isfile(filepath):
            with open(filepath, 'w') as f:
                f.close()
        if not names_only:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Writing course content(s) to '%s.txt'n" % (course_name))
        if names_only:
            sys.stdout.write(
                fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Writing course lecture names to '%s-names-only.txt'n" % (course_name))
        for chapter in chapters:
            chapter_id = chapter.id
            chapter_title = chapter.title
            lectures = chapter.get_lectures()
            lectures_count = chapter.lectures
            for lecture in lectures:
                lecture_id = lecture.id
                lecture_streams = lecture.streams
                lecture_best = lecture.getbest()
                lecture_assets = lecture.assets
                lecture_subtitles = lecture.subtitles
                if quality:
                    index = 0
                    while index < len(lecture_streams):
                        dimension = int(lecture_streams[index].dimention[1])
                        if dimension == quality:
                            lecture_best = lecture_streams[index]
                            break
                        index += 1
                    if not lecture_best:
                        lecture_best = lecture_best
                if caption_only and not skip_captions:
                    if lecture_subtitles:
                        for subtitle in lecture_subtitles:
                            self._write_to_file(filepath=course_path, lecture=subtitle, names_only=names_only, unsafe=unsafe)
                    if lecture_assets:
                        for asset in lecture_assets:
                            self._write_to_file(filepath=course_path, lecture=asset, names_only=names_only, unsafe=unsafe)
                elif skip_captions and not caption_only:
                    if lecture_best:
                        self._write_to_file(filepath=course_path, lecture=lecture_best, names_only=names_only, unsafe=unsafe)
                    if lecture_assets:
                        for asset in lecture_assets:
                            self._write_to_file(filepath=course_path, lecture=asset, names_only=names_only, unsafe=unsafe)
                else:
                    if lecture_best:
                        self._write_to_file(filepath=course_path, lecture=lecture_best, names_only=names_only, unsafe=unsafe)
                    if lecture_assets:
                        for asset in lecture_assets:
                            self._write_to_file(filepath=course_path, lecture=asset, names_only=names_only, unsafe=unsafe)
                    if lecture_subtitles:
                        for subtitle in lecture_subtitles:
                            self._write_to_file(filepath=course_path, lecture=subtitle, names_only=names_only, unsafe=unsafe)
        if not names_only:
            sys.stdout.write(
                fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Written successfully under '{name}.txt'.n".format(name=course_path))
        if names_only:
            sys.stdout.write(
                fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Written successfully under '{name}-names-only.txt'.n".format(
                    name=course_path))

    def course_list_down(self, chapter_number='', lecture_number='', unsafe=False):
        if not self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login as " + fm + sb + "(%s)" % (
                self.username) + fg + sb + "...n")
        if self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login using cookies ...n")
        course = udemy.course(url=self.url, username=self.username, password=self.password, cookies=self.cookies)
        course_id = course.id
        course_name = course.title
        total_lectures = course.lectures
        total_chapters = course.chapters
        chapters = course.get_chapters()
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Course " + fb + sb + "'%s'.n" % (course_name))
        sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sd + "Chapter(s) (%s).n" % (total_chapters))
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (total_lectures))
        if chapter_number and chapter_number > 0 and chapter_number <= total_chapters:
            chapter = chapters[chapter_number - 1]
            chapter_id = chapter.id
            chapter_title = chapter.title if not unsafe else "%02d" % (int(chapter.index))
            lectures = chapter.get_lectures()
            lectures_count = chapter.lectures
            sys.stdout.write('n' + fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%s-%s)n" % (chapter_title, chapter_id))
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (lectures_count))
            if lecture_number and lecture_number > 0 and lecture_number <= lectures_count:
                lecture = lectures[lecture_number - 1]
                lecture_id = lecture.id
                lecture_streams = lecture.streams
                lecture_best = lecture.getbest()
                lecture_assets = lecture.assets
                lecture_subtitles = lecture.subtitles
                if lecture_streams:
                    sys.stdout.write(fc + sd + "     - " + fy + sb + "duration   : " + fm + sb + str(lecture.duration) + fy + sb + ".n")
                    sys.stdout.write(fc + sd + "     - " + fy + sb + "Lecture id : " + fm + sb + str(lecture_id) + fy + sb + ".n")
                    for stream in lecture_streams:
                        content_length = stream.get_filesize()
                        if content_length != 0:
                            if content_length <= 1048576.00:
                                size = round(float(content_length) / 1024.00, 2)
                                sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                in_MB = 'KB' if size < 1024.00 else 'MB'
                            else:
                                size = round(float(content_length) / 1048576, 2)
                                sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                in_MB = "MB " if size < 1024.00 else 'GB '
                            if lecture_best.dimention[1] == stream.dimention[1]:
                                in_MB = in_MB + fc + sb + "(Best)" + fg + sd
                            sys.stdout.write(
                                't- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(stream), stream.dimention[1] + 'p', sz, in_MB, fy, sb))
                if lecture_assets:
                    for asset in lecture_assets:
                        if asset.mediatype != 'external_link':
                            content_length = asset.get_filesize()
                            if content_length != 0:
                                if content_length <= 1048576.00:
                                    size = round(float(content_length) / 1024.00, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = 'KB' if size < 1024.00 else 'MB'
                                else:
                                    size = round(float(content_length) / 1048576, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = "MB " if size < 1024.00 else 'GB '
                                sys.stdout.write('t- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(asset), asset.extension, sz, in_MB, fy, sb))
                if lecture_subtitles:
                    for subtitle in lecture_subtitles:
                        content_length = subtitle.get_filesize()
                        if content_length != 0:
                            if content_length <= 1048576.00:
                                size = round(float(content_length) / 1024.00, 2)
                                sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                in_MB = 'KB' if size < 1024.00 else 'MB'
                            else:
                                size = round(float(content_length) / 1048576, 2)
                                sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                in_MB = "MB " if size < 1024.00 else 'GB '
                            sys.stdout.write('t- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(subtitle), subtitle.extension, sz, in_MB, fy, sb))
            else:
                for lecture in lectures:
                    lecture_id = lecture.id
                    lecture_streams = lecture.streams
                    lecture_best = lecture.getbest()
                    lecture_assets = lecture.assets
                    lecture_subtitles = lecture.subtitles
                    if lecture_streams:
                        sys.stdout.write(fc + sd + "     - " + fy + sb + "duration   : " + fm + sb + str(lecture.duration) + fy + sb + ".n")
                        sys.stdout.write(fc + sd + "     - " + fy + sb + "Lecture id : " + fm + sb + str(lecture_id) + fy + sb + ".n")
                        for stream in lecture_streams:
                            content_length = stream.get_filesize()
                            if content_length != 0:
                                if content_length <= 1048576.00:
                                    size = round(float(content_length) / 1024.00, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = 'KB' if size < 1024.00 else 'MB'
                                else:
                                    size = round(float(content_length) / 1048576, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = "MB " if size < 1024.00 else 'GB '
                                if lecture_best.dimention[1] == stream.dimention[1]:
                                    in_MB = in_MB + fc + sb + "(Best)" + fg + sd
                                sys.stdout.write(
                                    't- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(stream), stream.dimention[1] + 'p', sz, in_MB, fy, sb))
                    if lecture_assets:
                        for asset in lecture_assets:
                            if asset.mediatype != 'external_link':
                                content_length = asset.get_filesize()
                                if content_length != 0:
                                    if content_length <= 1048576.00:
                                        size = round(float(content_length) / 1024.00, 2)
                                        sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                        in_MB = 'KB' if size < 1024.00 else 'MB'
                                    else:
                                        size = round(float(content_length) / 1048576, 2)
                                        sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                        in_MB = "MB " if size < 1024.00 else 'GB '
                                    sys.stdout.write('t- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(asset), asset.extension, sz, in_MB, fy, sb))
                    if lecture_subtitles:
                        for subtitle in lecture_subtitles:
                            content_length = subtitle.get_filesize()
                            if content_length != 0:
                                if content_length <= 1048576.00:
                                    size = round(float(content_length) / 1024.00, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = 'KB' if size < 1024.00 else 'MB'
                                else:
                                    size = round(float(content_length) / 1048576, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = "MB " if size < 1024.00 else 'GB '
                                sys.stdout.write(
                                    't- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(subtitle), subtitle.extension, sz, in_MB, fy, sb))
        else:
            for chapter in chapters:
                chapter_id = chapter.id
                chapter_title = chapter.title if not unsafe else "%02d" % (int(chapter.index))
                lectures = chapter.get_lectures()
                lectures_count = chapter.lectures
                sys.stdout.write('n' + fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%s-%s)n" % (chapter_title, chapter_id))
                sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (lectures_count))
                for lecture in lectures:
                    lecture_id = lecture.id
                    lecture_streams = lecture.streams
                    lecture_best = lecture.getbest()
                    lecture_assets = lecture.assets
                    lecture_subtitles = lecture.subtitles
                    if lecture_streams:
                        sys.stdout.write(fc + sd + "     - " + fy + sb + "duration   : " + fm + sb + str(lecture.duration) + fy + sb + ".n")
                        sys.stdout.write(fc + sd + "     - " + fy + sb + "Lecture id : " + fm + sb + str(lecture_id) + fy + sb + ".n")
                        for stream in lecture_streams:
                            content_length = stream.get_filesize()
                            if content_length != 0:
                                if content_length <= 1048576.00:
                                    size = round(float(content_length) / 1024.00, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = 'KB' if size < 1024.00 else 'MB'
                                else:
                                    size = round(float(content_length) / 1048576, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = "MB " if size < 1024.00 else 'GB '
                                if lecture_best.dimention[1] == stream.dimention[1]:
                                    in_MB = in_MB + fc + sb + "(Best)" + fg + sd
                                sys.stdout.write(
                                    't- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(stream), stream.dimention[1] + 'p', sz, in_MB, fy, sb))
                    if lecture_assets:
                        for asset in lecture_assets:
                            if asset.mediatype != 'external_link':
                                content_length = asset.get_filesize()
                                if content_length != 0:
                                    if content_length <= 1048576.00:
                                        size = round(float(content_length) / 1024.00, 2)
                                        sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                        in_MB = 'KB' if size < 1024.00 else 'MB'
                                    else:
                                        size = round(float(content_length) / 1048576, 2)
                                        sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                        in_MB = "MB " if size < 1024.00 else 'GB '
                                    sys.stdout.write('t- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(asset), asset.extension, sz, in_MB, fy, sb))
                    if lecture_subtitles:
                        for subtitle in lecture_subtitles:
                            content_length = subtitle.get_filesize()
                            if content_length != 0:
                                if content_length <= 1048576.00:
                                    size = round(float(content_length) / 1024.00, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = 'KB' if size < 1024.00 else 'MB'
                                else:
                                    size = round(float(content_length) / 1048576, 2)
                                    sz = format(size if size < 1024.00 else size / 1024.00, '.2f')
                                    in_MB = "MB " if size < 1024.00 else 'GB '
                                sys.stdout.write(
                                    't- ' + fg + sd + "{:<22} {:<8}{}{}{}{}n".format(str(subtitle), subtitle.extension, sz, in_MB, fy, sb))

    def download_assets(self, lecture_assets='', filepath='', unsafe=False):
        if lecture_assets:
            for assets in lecture_assets:
                title = assets.filename if not unsafe else "%s" % (assets)
                mediatype = assets.mediatype
                if mediatype == "external_link":
                    assets.download(filepath=filepath, unsafe=unsafe, quiet=True, callback=self.show_progress)
                else:
                    sys.stdout.write(fc + sd + "n[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Downloading asset(s)n")
                    sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Downloading (%s)n" % (title))
                    try:
                        retval = assets.download(filepath=filepath, unsafe=unsafe, quiet=True, callback=self.show_progress)
                    except KeyboardInterrupt:
                        sys.stdout.write(fc + sd + "n[" + fr + sb + "-" + fc + sd + "] : " + fr + sd + "User Interrupted..n")
                        sys.exit(0)
                    else:
                        msg = retval.get('msg')
                        if msg == 'already downloaded':
                            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Asset : '%s' " % (
                                title) + fy + sb + "(already downloaded).n")
                        elif msg == 'download':
                            sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sd + "Downloaded  (%s)n" % (title))
                        else:
                            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Asset : '%s' " % (
                                title) + fc + sb + "(download skipped).n")
                            sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sd + "{}n".format(msg))

    def download_subtitles(self, lecture_subtitles='', language='', filepath='', unsafe=False):
        if language:
            _lecture_subtitles = [i for i in lecture_subtitles if i.language == language]
            if _lecture_subtitles:
                lecture_subtitles = _lecture_subtitles
        if lecture_subtitles:
            for subtitles in lecture_subtitles:
                title = subtitles.title + '-' + subtitles.language if not unsafe else "%s" % (subtitles)
                if not unsafe:
                    filename = "%s%s" % (filepath, subtitles.filename) if os.name == 'nt' else "%s/%s" % (filepath, subtitles.filename)
                if unsafe:
                    filename = u"%s%s" % (filepath, subtitles.unsafe_filename) if os.name == 'nt' else u"%s/%s" % (
                        filepath, subtitles.unsafe_filename)

                sys.stdout.write(fc + sd + "n[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Downloading subtitle(s)n")
                sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Downloading (%s)n" % (title))

                try:
                    retval = subtitles.download(filepath=filepath, unsafe=unsafe, quiet=True, callback=self.show_progress)
                except KeyboardInterrupt:
                    sys.stdout.write(fc + sd + "n[" + fr + sb + "-" + fc + sd + "] : " + fr + sd + "User Interrupted..n")
                    sys.exit(0)
                else:
                    msg = retval.get('msg')
                    if msg == 'already downloaded':
                        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Subtitle : '%s' " % (
                            title) + fy + sb + "(already downloaded).n")
                        self.convert(filename=filename)
                    elif msg == 'download':
                        sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sd + "Downloaded  (%s)n" % (title))
                        self.convert(filename=filename)
                    else:
                        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Subtitle : '%s' " % (
                            title) + fc + sb + "(download skipped).n")
                        sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sd + "{}n".format(msg))

    def download_lectures(self, lecture_best='', lecture_title='', inner_index='', lectures_count='', filepath='', unsafe=False):
        if lecture_best:
            sys.stdout.write(
                fc + sd + "n[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) : ({index} of {total})n".format(index=inner_index,
                                                                                                                          total=lectures_count))
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Downloading (%s)n" % (lecture_title))
            try:
                retval = lecture_best.download(filepath=filepath, unsafe=unsafe, quiet=True, callback=self.show_progress)
            except KeyboardInterrupt:
                sys.stdout.write(fc + sd + "n[" + fr + sb + "-" + fc + sd + "] : " + fr + sd + "User Interrupted..n")
                sys.exit(0)
            else:
                msg = retval.get('msg')
                if msg == 'already downloaded':
                    if not unsafe:
                        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture : '%s' " % (
                            lecture_title) + fy + sb + "(already downloaded).n")
                    if unsafe:
                        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "'%s' " % (
                            lecture_title) + fy + sb + "(already downloaded).n")
                elif msg == 'download':
                    sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sd + "Downloaded  (%s)n" % (lecture_title))
                else:
                    if not unsafe:
                        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture : '%s' " % (
                            lecture_title) + fc + sb + "(download skipped).n")
                    if unsafe:
                        sys.stdout.write(
                            fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "'%s' " % (lecture_title) + fc + sb + "(download skipped).n")
                    sys.stdout.write(fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sd + "{}n".format(msg))

    def download_captions_only(self, lecture_subtitles='', language='', lecture_assets='', filepath='', unsafe=False):
        if lecture_subtitles:
            self.download_subtitles(lecture_subtitles=lecture_subtitles, language=language, filepath=filepath, unsafe=unsafe)
        if lecture_assets:
            self.download_assets(lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)

    def download_lectures_only(self, lecture_best='', lecture_title='', inner_index='', lectures_count='', lecture_assets='', filepath='',
                               unsafe=False):
        if lecture_best:
            self.download_lectures(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index, lectures_count=lectures_count,
                                   filepath=filepath, unsafe=unsafe)
        if lecture_assets:
            self.download_assets(lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)

    def download_lectures_and_captions(self, lecture_best='', lecture_title='', inner_index='', lectures_count='', lecture_subtitles='', language='',
                                       lecture_assets='', filepath='', unsafe=False):
        if lecture_best:
            self.download_lectures(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index, lectures_count=lectures_count,
                                   filepath=filepath, unsafe=unsafe)
        if lecture_subtitles:
            self.download_subtitles(lecture_subtitles=lecture_subtitles, language=language, filepath=filepath, unsafe=unsafe)
        if lecture_assets:
            self.download_assets(lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)

    def course_download(self, path='', quality='', language='', caption_only=False, skip_captions=False, unsafe=False):
        print('\bStarting...')
        if not self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login as " + fm + sb + "(%s)" % (
                self.username) + fg + sb + "...n")
        if self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login using cookies ...n")
        course = get_course(url=self.url, username=self.username, password=self.password, cookies=self.cookies)
        course_id = course.id
        course_name = course.title
        print('-----------------------------------------------------')
        print(course_id, course_name)
        print('-----------------------------------------------------')
        chapters = course.get_chapters()
        total_lectures = course.lectures
        total_chapters = course.chapters
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Course " + fb + sb + "'%s'.n" % (course_name))
        sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sd + "Chapter(s) (%s).n" % (total_chapters))
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (total_lectures))
        if path:
            if '~' in path:
                path = os.path.expanduser(path)
            course_path = "%s%s" % (path, course_name) if os.name == 'nt' else "%s/%s" % (path, course_name)
        else:
            path = os.getcwd()
            course_path = "%s%s" % (path, course_name) if os.name == 'nt' else "%s/%s" % (path, course_name)
        for chapter in chapters:
            chapter_id = chapter.id
            chapter_index = chapter.index
            chapter_title = chapter.title
            lectures = chapter.get_lectures()
            lectures_count = chapter.lectures
            if unsafe:
                filepath = u"%s%s" % (course_path, chapter.unsafe_title) if os.name == 'nt' else u"%s/%s" % (course_path, chapter.unsafe_title)
            if not unsafe:
                filepath = "%s%s" % (course_path, chapter_title) if os.name == 'nt' else "%s/%s" % (course_path, chapter_title)
            try:
                os.makedirs(filepath)
            except Exception as e:
                pass
            sys.stdout.write(fc + sd + "n[" + fm + sb + "*" + fc + sd + "] : " + fm + sb + "Downloading chapter : ({index} of {total})n".format(
                index=chapter_index, total=total_chapters))
            if not unsafe:
                sys.stdout.write(fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%s)n" % (chapter_title))
                sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Found (%s) lectures ...n" % (lectures_count))
            if unsafe:
                sys.stdout.write(fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%02d-%s)n" % (int(chapter_index), chapter_id))
                sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (lectures_count))
            inner_index = 1
            for lecture in lectures:
                lecture_id = lecture.id
                lecture_index = lecture.index
                lecture_title = lecture.title if not unsafe else "Lecture id : %s" % (lecture_id)
                lecture_assets = lecture.assets
                lecture_subtitles = lecture.subtitles
                lecture_best = lecture.getbest()
                lecture_streams = lecture.streams
                if caption_only and not skip_captions:
                    self.download_captions_only(lecture_subtitles=lecture_subtitles, language=language, lecture_assets=lecture_assets,
                                                filepath=filepath, unsafe=unsafe)
                elif skip_captions and not caption_only:
                    if quality:
                        index = 0
                        while index < len(lecture_streams):
                            dimension = int(lecture_streams[index].dimention[1])
                            if dimension == quality:
                                lecture_best = lecture_streams[index]
                                break
                            index += 1
                        if not lecture_best:
                            lecture_best = lecture_best
                    if lecture.html:
                        lecture.dump(filepath=filepath, unsafe=unsafe)
                    self.download_lectures_only(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index,
                                                lectures_count=lectures_count, lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                else:
                    if quality:
                        index = 0
                        while index < len(lecture_streams):
                            dimension = int(lecture_streams[index].dimention[1])
                            if dimension == quality:
                                lecture_best = lecture_streams[index]
                                break
                            index += 1
                        if not lecture_best:
                            lecture_best = lecture_best
                    if lecture.html:
                        lecture.dump(filepath=filepath, unsafe=unsafe)
                    self.download_lectures_and_captions(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index,
                                                        lectures_count=lectures_count, lecture_subtitles=lecture_subtitles, language=language,
                                                        lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                inner_index += 1

    def chapter_download(self, chapter_number='', chapter_start='', chapter_end='', lecture_number='', lecture_start='', lecture_end='', path='',
                         quality='', language='', caption_only=False, skip_captions=False, unsafe=False):
        if not self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login as " + fm + sb + "(%s)" % (
                self.username) + fg + sb + "...n")
        if self.cookies:
            sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Trying to login using cookies ...n")
        course = udemy.course(url=self.url, username=self.username, password=self.password, cookies=self.cookies)
        course_id = course.id
        course_name = course.title
        chapters = course.get_chapters()
        total_lectures = course.lectures
        total_chapters = course.chapters
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sb + "Course " + fb + sb + "'%s'.n" % (course_name))
        sys.stdout.write(fc + sd + "[" + fm + sb + "+" + fc + sd + "] : " + fg + sd + "Chapter(s) (%s).n" % (total_chapters))
        sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (total_lectures))
        if path:
            if '~' in path:
                path = os.path.expanduser(path)
            course_path = "%s%s" % (path, course_name) if os.name == 'nt' else "%s/%s" % (path, course_name)
        else:
            path = os.getcwd()
            course_path = "%s%s" % (path, course_name) if os.name == 'nt' else "%s/%s" % (path, course_name)
        _lectures_start, _lectures_end = lecture_start, lecture_end
        if chapter_start and not chapter_end:
            chapter_end = total_chapters
        if chapter_number and chapter_number > 0 and chapter_number <= total_chapters:
            chapter = chapters[chapter_number - 1]
            if chapter:
                chapter_id = chapter.id
                chapter_index = chapter.index
                chapter_title = chapter.title
                lectures = chapter.get_lectures()
                lectures_count = chapter.lectures
                if lecture_end and lecture_end > lectures_count:
                    lecture_end = lectures_count

                if unsafe:
                    filepath = u"%s%s" % (course_path, chapter.unsafe_title) if os.name == 'nt' else u"%s/%s" % (course_path, chapter.unsafe_title)
                if not unsafe:
                    filepath = "%s%s" % (course_path, chapter_title) if os.name == 'nt' else "%s/%s" % (course_path, chapter_title)
                try:
                    os.makedirs(filepath)
                except Exception as e:
                    pass
                sys.stdout.write(
                    fc + sd + "n[" + fm + sb + "*" + fc + sd + "] : " + fm + sb + "Downloading chapter : ({index})n".format(index=chapter_index))
                if not unsafe:
                    sys.stdout.write(fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%s)n" % (chapter_title))
                    sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Found (%s) lectures ...n" % (lectures_count))
                if unsafe:
                    sys.stdout.write(
                        fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%02d-%s)n" % (int(chapter_index), chapter_id))
                    sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (lectures_count))
                lecture_start = _lectures_start
                lecture_end = lectures_count if lecture_start and not lecture_end else _lectures_end
                if lecture_number and lecture_number > 0 and lecture_number <= lectures_count:
                    lecture = lectures[lecture_number - 1]
                    lecture_id = lecture.id
                    lecture_index = lecture.index
                    lecture_title = lecture.title if not unsafe else "Lecture id : %s" % (lecture_id)
                    lecture_assets = lecture.assets
                    lecture_subtitles = lecture.subtitles
                    lecture_best = lecture.getbest()
                    lecture_streams = lecture.streams
                    if caption_only and not skip_captions:
                        self.download_captions_only(lecture_subtitles=lecture_subtitles, language=language, lecture_assets=lecture_assets,
                                                    filepath=filepath, unsafe=unsafe)
                    elif skip_captions and not caption_only:
                        if quality:
                            index = 0
                            while index < len(lecture_streams):
                                dimension = int(lecture_streams[index].dimention[1])
                                if dimension == quality:
                                    lecture_best = lecture_streams[index]
                                    break
                                index += 1
                            if not lecture_best:
                                lecture_best = lecture_best
                        if lecture.html:
                            lecture.dump(filepath=filepath)
                        self.download_lectures_only(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_number,
                                                    lectures_count=lectures_count, lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                    else:
                        if quality:
                            index = 0
                            while index < len(lecture_streams):
                                dimension = int(lecture_streams[index].dimention[1])
                                if dimension == quality:
                                    lecture_best = lecture_streams[index]
                                    break
                                index += 1
                            if not lecture_best:
                                lecture_best = lecture_best
                        if lecture.html:
                            lecture.dump(filepath=filepath)
                        self.download_lectures_and_captions(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_number,
                                                            lectures_count=lectures_count, lecture_subtitles=lecture_subtitles, language=language,
                                                            lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                elif lecture_start and lecture_start > 0 and lecture_start <= lecture_end and lecture_end <= lectures_count:
                    while lecture_start <= lecture_end:
                        lecture = lectures[lecture_start - 1]
                        lecture_id = lecture.id
                        lecture_index = lecture.index
                        lecture_title = lecture.title if not unsafe else "Lecture id : %s" % (lecture_id)
                        lecture_assets = lecture.assets
                        lecture_subtitles = lecture.subtitles
                        lecture_best = lecture.getbest()
                        lecture_streams = lecture.streams
                        if caption_only and not skip_captions:
                            self.download_captions_only(lecture_subtitles=lecture_subtitles, language=language, lecture_assets=lecture_assets,
                                                        filepath=filepath, unsafe=unsafe)
                        elif skip_captions and not caption_only:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            if lecture.html:
                                lecture.dump(filepath=filepath, unsafe=unsafe)
                            self.download_lectures_only(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_start,
                                                        lectures_count=lecture_end, lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                        else:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            if lecture.html:
                                lecture.dump(filepath=filepath, unsafe=unsafe)
                            self.download_lectures_and_captions(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_start,
                                                                lectures_count=lecture_end, lecture_subtitles=lecture_subtitles, language=language,
                                                                lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                        lecture_start += 1
                else:
                    inner_index = 1
                    for lecture in lectures:
                        lecture_id = lecture.id
                        lecture_index = lecture.index
                        lecture_title = lecture.title if not unsafe else "Lecture id : %s" % (lecture_id)
                        lecture_assets = lecture.assets
                        lecture_subtitles = lecture.subtitles
                        lecture_best = lecture.getbest()
                        lecture_streams = lecture.streams
                        if caption_only and not skip_captions:
                            self.download_captions_only(lecture_subtitles=lecture_subtitles, language=language, lecture_assets=lecture_assets,
                                                        filepath=filepath, unsafe=unsafe)
                        elif skip_captions and not caption_only:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            if lecture.html:
                                lecture.dump(filepath=filepath, unsafe=unsafe)
                            self.download_lectures_only(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index,
                                                        lectures_count=lectures_count, lecture_assets=lecture_assets, filepath=filepath,
                                                        unsafe=unsafe)
                        else:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            if lecture.html:
                                lecture.dump(filepath=filepath, unsafe=unsafe)
                            self.download_lectures_and_captions(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index,
                                                                lectures_count=lectures_count, lecture_subtitles=lecture_subtitles, language=language,
                                                                lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                        inner_index += 1
        elif chapter_start and chapter_start > 0 and chapter_start <= chapter_end and chapter_end <= total_chapters:
            while chapter_start <= chapter_end:
                chapter = chapters[chapter_start - 1]
                chapter_id = chapter.id
                chapter_index = chapter.index
                chapter_title = chapter.title
                lectures = chapter.get_lectures()
                lectures_count = chapter.lectures
                if unsafe:
                    filepath = u"%s%s" % (course_path, chapter.unsafe_title) if os.name == 'nt' else u"%s/%s" % (course_path, chapter.unsafe_title)
                if not unsafe:
                    filepath = "%s%s" % (course_path, chapter_title) if os.name == 'nt' else "%s/%s" % (course_path, chapter_title)
                try:
                    os.makedirs(filepath)
                except Exception as e:
                    pass
                sys.stdout.write(fc + sd + "n[" + fm + sb + "*" + fc + sd + "] : " + fm + sb + "Downloading chapter : ({index} of {total})n".format(
                    index=chapter_start, total=chapter_end))
                if not unsafe:
                    sys.stdout.write(fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%s)n" % (chapter_title))
                    sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Found (%s) lectures ...n" % (lectures_count))
                if unsafe:
                    sys.stdout.write(
                        fc + sd + "[" + fw + sb + "+" + fc + sd + "] : " + fw + sd + "Chapter (%02d-%s)n" % (int(chapter_index), chapter_id))
                    sys.stdout.write(fc + sd + "[" + fm + sb + "*" + fc + sd + "] : " + fg + sd + "Lecture(s) (%s).n" % (lectures_count))
                lecture_start = _lectures_start
                lecture_end = lectures_count if lecture_start and not lecture_end else _lectures_end
                if lecture_number and lecture_number > 0 and lecture_number <= lectures_count:
                    lecture = lectures[lecture_number - 1]
                    lecture_id = lecture.id
                    lecture_index = lecture.index
                    lecture_title = lecture.title if not unsafe else "Lecture id : %s" % (lecture_id)
                    lecture_assets = lecture.assets
                    lecture_subtitles = lecture.subtitles
                    lecture_best = lecture.getbest()
                    lecture_streams = lecture.streams
                    if caption_only and not skip_captions:
                        self.download_captions_only(lecture_subtitles=lecture_subtitles, language=language, lecture_assets=lecture_assets,
                                                    filepath=filepath, unsafe=unsafe)
                    elif skip_captions and not caption_only:
                        if quality:
                            index = 0
                            while index < len(lecture_streams):
                                dimension = int(lecture_streams[index].dimention[1])
                                if dimension == quality:
                                    lecture_best = lecture_streams[index]
                                    break
                                index += 1
                            if not lecture_best:
                                lecture_best = lecture_best
                        self.download_lectures_only(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_number,
                                                    lectures_count=lectures_count, lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                    else:
                        if quality:
                            index = 0
                            while index < len(lecture_streams):
                                dimension = int(lecture_streams[index].dimention[1])
                                if dimension == quality:
                                    lecture_best = lecture_streams[index]
                                    break
                                index += 1
                            if not lecture_best:
                                lecture_best = lecture_best
                        self.download_lectures_and_captions(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_number,
                                                            lectures_count=lectures_count, lecture_subtitles=lecture_subtitles, language=language,
                                                            lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                elif lecture_start and lecture_start > 0 and lecture_start <= lecture_end and lecture_end <= lectures_count:
                    while lecture_start <= lecture_end:
                        lecture = lectures[lecture_start - 1]
                        lecture_id = lecture.id
                        lecture_index = lecture.index
                        lecture_title = lecture.title if not unsafe else "Lecture id : %s" % (lecture_id)
                        lecture_assets = lecture.assets
                        lecture_subtitles = lecture.subtitles
                        lecture_best = lecture.getbest()
                        lecture_streams = lecture.streams
                        if caption_only and not skip_captions:
                            self.download_captions_only(lecture_subtitles=lecture_subtitles, language=language, lecture_assets=lecture_assets,
                                                        filepath=filepath, unsafe=unsafe)
                        elif skip_captions and not caption_only:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            self.download_lectures_only(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_start,
                                                        lectures_count=lecture_end, lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                        else:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            self.download_lectures_and_captions(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=lecture_start,
                                                                lectures_count=lecture_end, lecture_subtitles=lecture_subtitles, language=language,
                                                                lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                        lecture_start += 1
                else:
                    inner_index = 1
                    for lecture in lectures:
                        lecture_id = lecture.id
                        lecture_index = lecture.index
                        lecture_title = lecture.title if not unsafe else "Lecture id : %s" % (lecture_id)
                        lecture_assets = lecture.assets
                        lecture_subtitles = lecture.subtitles
                        lecture_best = lecture.getbest()
                        lecture_streams = lecture.streams
                        if caption_only and not skip_captions:
                            self.download_captions_only(lecture_subtitles=lecture_subtitles, language=language, lecture_assets=lecture_assets,
                                                        filepath=filepath, unsafe=unsafe)
                        elif skip_captions and not caption_only:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            self.download_lectures_only(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index,
                                                        lectures_count=lectures_count, lecture_assets=lecture_assets, filepath=filepath,
                                                        unsafe=unsafe)
                        else:
                            if quality:
                                index = 0
                                while index < len(lecture_streams):
                                    dimension = int(lecture_streams[index].dimention[1])
                                    if dimension == quality:
                                        lecture_best = lecture_streams[index]
                                        break
                                    index += 1
                                if not lecture_best:
                                    lecture_best = lecture_best
                            self.download_lectures_and_captions(lecture_best=lecture_best, lecture_title=lecture_title, inner_index=inner_index,
                                                                lectures_count=lectures_count, lecture_subtitles=lecture_subtitles, language=language,
                                                                lecture_assets=lecture_assets, filepath=filepath, unsafe=unsafe)
                        inner_index += 1
                chapter_start += 1
        else:
            if not chapter_end and not chapter_number and not chapter_start:
                sys.stdout.write(
                    fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Argument(s) are missing : Chapter(s) range or chapter(s) number is required.n")
            elif chapter_end and chapter_end > total_chapters or chapter_number and chapter_number > total_chapters:
                sys.stdout.write(
                    fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Chapter(s) Range exceeded : Chapter(s) ending or chapter(s) number is out of rangen")
            elif chapter_start and chapter_start > chapter_end:
                sys.stdout.write(
                    fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Chapter(s) Range exception : Chapter(s) starting point cannot be greater than chapter(s) ending pointn")
            elif chapter_end and not chapter_start:
                sys.stdout.write(
                    fc + sd + "[" + fr + sb + "-" + fc + sd + "] : " + fr + sb + "Argument(s) are missing : Chapter(s) range starting point is missing ..n")
            sys.stdout.write(
                fc + sd + "[" + fy + sb + "i" + fc + sd + "] : " + fw + sb + "Chapter(s) number or range should be in between ({start} to {end}).n".format(
                    start=1, end=total_chapters))
            sys.exit(0)

clear_output()
print('prepration is complete')


prepration is complete


In [0]:
#@title Add Udemy links in the `courses` list

courses = [
    "https://www.udemy.com/course/skillshare-course-marketing/",
]

language = ''         # left blank if you want all language or put some e.g. en

try:
    for course in courses:
        udemy = Udemy(url=course, username=username, password=password, cookies=cookies)
        udemy.course_download(path=output, quality=quality, language=language)
except Exception as e:
    print(e.args)